# SARS-CoV-2 data processing

This notebook performs four main functions:

- [Merging sequences and metadata](#merge)  
- [Selecting and exporting sequences for analysis](#select)  
- [Finding groups of linked mutations and classifying mutations as synonymous or nonsynonymous](#link)
- [Inferring selection coefficients for various different scenarios](#infer)

In [1]:
import os, sys
from imp import reload
import datetime as dt
import shutil
print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import pandas as pd
print('pandas version %s' % pd.__version__)

import matplotlib
import matplotlib.pyplot as plt
print('matplotlib version %s' % matplotlib.__version__)

import scipy
from scipy import linalg
print('scipy version %s' % scipy.__version__)

from tqdm import tqdm

# GLOBAL VARIABLES

# sequence related variables
NUC = ['-', 'A', 'C', 'G', 'T']

# local directories
SARS_DIR     = os.getcwd()
#INF_SCRIPTS  = 'inf-scripts'
DATA_DATE    = '2021-08-14'
DATA_DIR     = os.path.join(SARS_DIR, 'data')
SCRIPT_DIR   = os.path.join(SARS_DIR, 'processing-files')
INF_SCRIPTS  = SCRIPT_DIR
INF_DIR      = DATA_DIR

METADATA_COMP    = os.path.join(SARS_DIR, f'metadata_tsv_2021_12_06.tar.xz')          # compressed metadata file from GISAID
METADATA_FILE    = os.path.join(SARS_DIR, f'metadata-{DATA_DATE}', 'metadata.tsv')    # decompressed metadata file from GISAID
METADATA_DIR     = os.path.join(SARS_DIR, f'metadata-{DATA_DATE}')

METADATA_INTERIM = 'merged-interim.csv'
MERGED_FINAL     = 'merged-final.csv'

# cluster directories
USER_NAME  = 'blee098'
EMAIL_EXT  = '@ucr.edu'    # email extension for account attached to cluster
SSH_HOME   = 'blee098@cluster.hpcc.ucr.edu:'
SSH_DATA   = os.path.join('/rhome', USER_NAME, 'bigdata', 'SARS-CoV-2-Data')
SCRATCH    = os.path.join(SSH_DATA, 'scratch')

# metadata variables
METADATA_COLS    = [   'accession', 'virus_name',            'date', 'location',             'location_additional']
METADATA_OCOLS   = ['Accession ID', 'Virus name', 'Collection date', 'Location', 'Additional location information']
METADATA_XFORM   = [           str,    str.lower,               str,  str.lower,                         str.lower]
REF_TAG          = 'EPI_ISL_402125'

# SEQUENCE PROCESSING GLOBAL VARIABLES

START_IDX    = 0
END_IDX      = -1
MAX_GAP_NUM  = 20000
MAX_GAP_FREQ = 0.99
MIN_SEQS     = 0
MAX_DT       = 999

import data_processing as dp

python version 3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]
numpy version 1.18.5
pandas version 1.0.5
matplotlib version 3.2.2
scipy version 1.5.0


In [ ]:
#for directory in [INF_DIR, DATA_DATE, SARS_DIR, INF_SCRIPTS, DATA_DIR, SCRIPT_DIR]:
for directory in [INF_DIR, SARS_DIR, INF_SCRIPTS, DATA_DIR, SCRIPT_DIR]:
    if not os.path.exists(directory):
        os.mkdir(directory)

<a id='merge'></a>
# Merging sequences and metadata

### Extracting the accessions, virus name, date, and location information from metadata file

In [217]:
print(f'mkdir {METADATA_DIR} &&')
print(f'tar -xf {METADATA_COMP} -C {METADATA_DIR}')

In [281]:
accessions = set()
header     = ','.join(METADATA_COLS)

use_cols = ['Accession ID', 'Virus name', 'Collection date', 'Location', 
            'Additional location information', 'Host']
df = pd.read_csv('%s' % METADATA_FILE, sep='\t', usecols=use_cols)
f = open('%s' % METADATA_INTERIM, 'w')
f.write('%s\n' % header)
for df_iter, df_entry in tqdm(df.iterrows(), total=df.shape[0]):
    #if df_iter % 10000 == 0:
    #    print(df_inter / len(df))
    entry = []
    unique_id  = False
    valid_date = False
        
    # check that host is human
    if 'Host' in df_entry:
        if df_entry.Host!='Human' and df_entry.Host!='human':
            continue
        
    # get accession
    acc_idx = METADATA_COLS.index('accession')
    if 'Accession ID' not in df_entry:
        continue
    xformed = METADATA_XFORM[acc_idx](df_entry[METADATA_OCOLS[acc_idx]])
    acc     = METADATA_XFORM[acc_idx](df_entry[METADATA_OCOLS[acc_idx]])

    if xformed not in accessions:
        #accessions.append(xformed)
        accessions.add(xformed)
        entry.append(xformed)
        unique_id = True
    #else:
        #print('Excluding %s (duplicate accession number)' % xformed)
        
    # get virus name 
    if 'Virus name' in df_entry:
        name_idx = METADATA_COLS.index('virus_name')
        xformed  = METADATA_XFORM[name_idx](df_entry[METADATA_OCOLS[name_idx]])
        xformed  = xformed.replace(',', ' ')
        entry.append(xformed)
    else:
        entry.append('NA')
            
    # get date
    if unique_id:
        date_idx = METADATA_COLS.index('date')
        xformed  = METADATA_XFORM[date_idx](df_entry[METADATA_OCOLS[date_idx]])
            
        try:
            dt.date.fromisoformat(xformed)
            valid_date = True
        except:
            valid_date = False
            #print('Excluding %s (date %s is incomplete)' % (entry[0], xformed))
        entry.append(xformed)
            
    # get location 
    if unique_id and valid_date:
        loc_idx = METADATA_COLS.index('location')
        xformed = METADATA_XFORM[loc_idx](df_entry[METADATA_OCOLS[loc_idx]])
        xformed = xformed.replace(',', ' ')
        entry.append(xformed)
        
        loc_add_idx = METADATA_COLS.index('location_additional')
        xformed     = METADATA_XFORM[loc_add_idx](str(df_entry[METADATA_OCOLS[loc_add_idx]]))
        xformed     = xformed.replace(',', ' ')
        entry.append(xformed)
        
    # save data
    if unique_id and valid_date:
        f.write('%s\n' % ','.join(entry))
            

# Add reference if not in the list of accessions
if REF_TAG not in accessions:
    entry = []
    for i in range(len(METADATA_COLS)):
        if METADATA_COLS[i]=='accession':
            entry.append(REF_TAG)
        elif METADATA_COLS[i]=='date':
            entry.append('2020-01-01')
        else:
            entry.append('reference')
    f.write('%s\n' % ','.join(entry))
f.close()

KeyboardInterrupt: 

### Merging sequence data in the GISAID alignment with metadata on the cluster

In [71]:
#np.save(os.path.join(SARS_DIR, 'date-ranges.npy'), DATE_RANGES)
msa_name         = 'msa_0814'
msa_file         = f'{msa_name}.fasta'
msa_path_cluster = os.path.join(SSH_DATA, msa_name, msa_name + '.fasta')
script_file      = 'merge-metadata-alignment.py'
job_file         = 'job-merge-alignment.sh'
job_str          = f"""#!/bin/bash

#SBATCH --time=4-0
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --job-name=merge
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH -e ./MPL/out/merge-error
#SBATCH -o ./MPL/out/merge-out
#SBATCH -p highmem

module purge
module load anaconda3

python {script_file} --meta_file {os.path.join(SSH_DATA, METADATA_INTERIM)} --msa_file {msa_path_cluster} -o {os.path.join(SSH_DATA, MERGED_FINAL[:MERGED_FINAL.find('.')])}
""" 

f = open(os.path.join(SARS_DIR, job_file), mode='w')
f.write('%s\n' % job_str)
f.close()

print('scp %s/%s %s &&' % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s/%s &&' % (SARS_DIR, METADATA_INTERIM, SSH_HOME, SSH_DATA))
print('scp %s/%s %s &&' % (SCRIPT_DIR, 'data_processing.py', SSH_HOME))
print('scp %s/%s %s '   % (SARS_DIR, job_file, SSH_HOME))
print('')

msa_file2 = f'{msa_name}.tar.xz'
msa_path2 = os.path.join(SARS_DIR, msa_file2)
print('scp %s %s/%s/%s' % (msa_path2, SSH_HOME, SSH_DATA, msa_file2))
print('')

# commands to execute on the cluster
print('tar -xf %s/%s -C %s &&' % (SSH_DATA, msa_file2, SSH_DATA))
print('mkdir -p ./MPL/out &&')
print('sbatch %s' % job_file)
print('')

scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/merge-metadata-alignment.py blee098@cluster.hpcc.ucr.edu: &&
scp /Users/brianlee/SARS-CoV-2-Data/merged-interim.csv blee098@cluster.hpcc.ucr.edu://rhome/blee098/bigdata/SARS-CoV-2-Data &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu: &&
scp /Users/brianlee/SARS-CoV-2-Data/job-merge-alignment.sh blee098@cluster.hpcc.ucr.edu: 

scp /Users/brianlee/SARS-CoV-2-Data/msa_0814.tar.xz blee098@cluster.hpcc.ucr.edu://rhome/blee098/bigdata/SARS-CoV-2-Data/msa_0814.tar.xz

tar -xf /rhome/blee098/bigdata/SARS-CoV-2-Data/msa_0814.tar.xz -C /rhome/blee098/bigdata/SARS-CoV-2-Data &&
mkdir -p ./MPL/out &&
sbatch job-merge-alignment.sh



### Checking sampling by region

In [400]:
# finding the number of sequences in each region

out_file      = f'regional-{DATA_DATE}'
script_file   = 'check-regional.py'
job_dir       = SSH_HOME + SSH_DATA
job_file      = 'job-check-regional.sh'
sars_dir_loc  = './bigdata/SARS-CoV-2-Data/'
job_str       = f"""#!/bin/bash

#SBATCH --time=0-3
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=100G
#SBATCH --job-name=regional-check
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH -e ./MPL/out/region-error
#SBATCH -o ./MPL/out/region-out

module purge
module load anaconda3

python {script_file} --alignment {os.path.join(SSH_DATA, MERGED_FINAL)} -o {os.path.join(SSH_DATA, out_file)}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), mode='w')
f.write('%s\n' % job_str)
f.close()

# transfer data processing scripts and job file to the cluster
print('scp %s/%s %s &&' % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s' % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

# run the job on the cluster
print('sbatch %s' % job_file)
print('')

# transfer output data back to local directory
print('scp %s/%s %s &&' % (job_dir, out_file+'.csv', SARS_DIR))

scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/check-regional.py blee098@cluster.hpcc.ucr.edu: &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-check-regional.sh blee098@cluster.hpcc.ucr.edu:

sbatch job-check-regional.sh

scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/regional-2021-08-14.csv /Users/brianlee/SARS-CoV-2-Data &&
scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/regional-2021-08-14-usa.csv /Users/brianlee/SARS-CoV-2-Data &&
scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/regional-2021-08-14-usa-sub.csv /Users/brianlee/SARS-CoV-2-Data &&
scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/regional-2021-08-14-uk.csv /Users/brianlee/SARS-CoV-2-Data &&
scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/regional-2021-08-14-uk-reg.csv /Users/brianlee/SARS-CoV-2-Data &&
scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/regional-20

In [15]:
pd.set_option('max_rows', 300)

In [ ]:
df = pd.read_csv(os.path.join(SARS_DIR, f'regional-{DATA_DATE}.csv'))
print(df['country'].value_counts())

In [ ]:
df_usa = df[df['country']=='united states']
print(df_usa['region'].value_counts())

# Extracting regions and time series for analysis
<a id='select'></a>

In [10]:
# Choosing regions to extract sequence data for

identifier = f'regions-{DATA_DATE}'

selected   =  [['south america', 'peru', None, None, None, None],
               ['north america', 'usa', 'alabama', None, None, None],
               ['north america', 'usa', 'arizona', None, None, None],
               ['north america', 'usa', 'arkansas', None, None, None],
               ['north america', 'usa', 'california',
                ['los angeles', 'los angeles county', 'orange', 'orange county', 'san diego', 'san diego county', 
                 'ventura', 'ventura county', 'san luis obispo', 'san luis obispo county', 'imperial', 'imperial county'], 
                None, None],
               ['north america', 'usa', 'colorado', None, None, None],
               ['north america', 'usa', 'connecticut', None, None, None],
               ['north america', 'usa', 'delaware', None, None, None],
               ['north america', 'usa', 'district of columbia', None, None, None],
               ['north america', 'usa', 'florida', None, None, None],
               ['north america', 'usa', 'georgia', None, None, None],
               ['north america', 'usa', 'hawaii', None, None, None],
               ['north america', 'usa', 'idaho', None, None, None],
               ['north america', 'usa', 'illinois', None, None, None],
               ['north america', 'usa', 'indiana', None, None, None],
               ['north america', 'usa', 'kansas', None, None, None],
               ['north america', 'usa', 'kentucky', None, None, None],
               ['north america', 'usa', 'louisiana', None, None, None],
               ['north america', 'usa', 'maine', None, None, None],
               ['north america', 'usa', 'maryland', None, None, None], 
               ['north america', 'usa', 'massachusetts', None, None, None],
               ['north america', 'usa', 'michigan', None, None, None],
               ['north america', 'usa', 'minnesota', None, None, None],
               ['north america', 'usa', 'missouri', None, None, None],
               ['north america', 'usa', 'montana', None, None, None],
               ['north america', 'usa', 'nebraska', None, None, None],
               ['north america', 'usa', 'nevada', None, None, None],
               ['north america', 'usa', 'new hampshire', None, None, None],
               ['north america', 'usa', 'new jersey', None, None, None],
               ['north america', 'usa', 'new mexico', None, None, None],
               ['north america', 'usa', 'new york', None, None, None],
               ['north america', 'usa', 'north carolina', None, None, None],
               ['north america', 'usa', 'north dakota', None, None, None],
               ['north america', 'usa', 'ohio', None, None, None],
               ['north america', 'usa', 'oregon', None, None, None], 
               ['north america', 'usa', 'pennsylvania', None, None, None],
               ['north america', 'usa', 'puerto rico', None, None, None],
               ['north america', 'usa', 'rhode island', None, None, None],
               ['north america', 'usa', 'south carolina', None, None, None],
               ['north america', 'usa', 'tennessee', None, None, None],
               ['north america', 'usa', 'texas', None, None, None],
               ['north america', 'usa', 'utah', None, None, None],
               ['north america', 'usa', 'virginia', None, None, None],
               ['north america', 'usa', 'washington', None, None, None], 
               ['north america', 'usa', 'west virginia', None, None, None],
               ['north america', 'usa', 'wisconsin', None, None, None],
               ['north america', 'usa', 'wyoming', None, None, None],
               ['europe', 'austria', None, None, None, None],
               ['europe', 'bangladesh', None, None, None, None],
               ['europe', 'belgium', None, None, None, None],
               ['europe', 'bulgaria', None, None, None, None],
               ['europe', 'czech republic', None, None, None, None], 
               ['europe', 'denmark', None, None, None, None],
               ['europe', 'estonia', None, None, None, None],
               ['europe', 'finland', None, None, None, None],
               ['europe', 'france', None, None, None, None], 
               ['europe', 'germany', None, None, None, None],
               ['europe', 'greece', None, None, None, None],
               ['europe', 'ireland', None, None, None, None],
               ['europe', 'israel', None, None, None, None],
               ['europe', 'italy', None, None, None, None],
               ['europe', 'jordan', None, None, None, None],
               ['europe', 'latvia', None, None, None, None],
               ['europe', 'lithuania', None, None, None, None], 
               ['europe', 'luxembourg', None, None, None, None],
               ['europe', 'netherlands', None, None, None, None],
               ['europe', 'norway', None, None, None, None],
               ['europe', 'poland', None, None, None, None], 
               ['europe', 'portugal', None, None, None, None],
               ['europe', 'romania', None, None, None, None],
               ['europe', 'slovakia', None, None, None, None],
               ['europe', 'slovenia', None, None, None, None],
               ['europe', 'spain', None, None, None, None],
               ['europe', 'sweden', None, None, None, None], 
               ['europe', 'switzerland', None, None, None, None],
               ['europe', 'united arab emirates', None, None, None, None],
               ['oceania', 'australia', None, None, None, None],
               ['asia', 'croatia', None, None, None, None],
               ['asia', 'china', None, None, None, None],
               ['asia', 'japan', None, None, None, None],
               ['asia', 'malaysia', None, None, None, None],
               ['asia', 'philippines', None, None, None, None],
               ['asia', 'saudi arabia', None, None, None, None],
               ['asia', 'thailand', None, None, None, None],
               ['asia', 'qatar', None, None, None, None],
               ['asia', 'united arab emirates', None, None, None, None],
               ['africa', 'kenya', None, None, None, None],
               ['africa', 'south africa', None, None, None, None]]
f = open(os.path.join(SARS_DIR, identifier + '.npy'), mode='w+b')
np.save(f, selected)
f.close()

identifier2 = f'regions-{DATA_DATE}b'
selected2   = [['europe', 'united kingdom', ['england', 'wales', 'scotland'], None, '2020-01-01', '2021-01-01'],
               ['europe', 'united kingdom', 'northern ireland', None, None, None]]
f = open(os.path.join(SARS_DIR, identifier2 + '.npy'), mode='w+b')
np.save(f, selected2)
f.close()

identifier3 = f'regions-{DATA_DATE}c'
selected3   = [['europe', 'united kingdom', ['england', 'wales', 'scotland'], None, '2021-01-01', '2021-03-01']]
f = open(os.path.join(SARS_DIR, identifier3 + '.npy'), mode='w+b')
np.save(f, selected3)
f.close()

identifier4 = f'regions-{DATA_DATE}d'
selected4   = [['europe', 'united kingdom', ['england', 'wales', 'scotland'], None, '2021-03-01', '2021-05-01']]
f = open(os.path.join(SARS_DIR, identifier4 + '.npy'), mode='w+b')
np.save(f, selected4)
f.close()

identifier5 = f'regions-{DATA_DATE}e'
selected5   = [['europe', 'united kingdom', ['england', 'wales', 'scotland'], None, '2021-05-01', '2021-07-01']]
f = open(os.path.join(SARS_DIR, identifier5 + '.npy'), mode='w+b')
np.save(f, selected5)
f.close()

identifier6 = f'regions-{DATA_DATE}f'
selected6   = [['europe', 'united kingdom', ['england', 'wales', 'scotland'], None, '2021-07-01', '2021-09-01']]
f = open(os.path.join(SARS_DIR, identifier6 + '.npy'), mode='w+b')
np.save(f, selected6)
f.close()

identifier7 = f'regions-{DATA_DATE}g'
selected7   = [['europe', 'united kingdom', ['england', 'wales', 'scotland'], None, '2021-09-01', '2021-11-01']]
f = open(os.path.join(SARS_DIR, identifier7 + '.npy'), mode='w+b')
np.save(f, selected7)
f.close()

In [11]:
# Transfer scripts, run processing, and extract files.
# Processing extracts region specific sequences and eliminates non-polymorphic sites to reduce required computational load of downstream analysis.
# Data is also filtered to eliminate sequences with many gaps and sites that are primarily gaps, to impute ambiguous nucleotides.
# Sequences are then time-ordered and formatted so that they can be read into the Inference scripts.
data_module   = 'data_processing.py'
data_mod_path = os.path.join(SCRIPT_DIR, data_module)
out_folder    = os.path.join(SSH_DATA, 'data')
input_file    = 'merged-final.csv'
script_file   = 'processing-multiallele.py'

job_file      = 'job-processing.sh'
job_str       = f"""#!/bin/bash 
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=900G
#SBATCH --time=2-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/genome-error
#SBATCH -o ./MPL/out/genome-out
#SBATCH -p highmem

module purge
module load anaconda3

python {script_file} --input_file {os.path.join(SSH_DATA, input_file)} -o {out_folder} --regions {identifier   + '.npy'} --find_syn_off --no_trim &&
python {script_file} --input_file {os.path.join(SSH_DATA, input_file)} -o {out_folder} --regions {identifier2  + '.npy'} --find_syn_off --no_trim && 
python {script_file} --input_file {os.path.join(SSH_DATA, input_file)} -o {out_folder} --regions {identifier3  + '.npy'} --find_syn_off --no_trim &&
python {script_file} --input_file {os.path.join(SSH_DATA, input_file)} -o {out_folder} --regions {identifier4  + '.npy'} --find_syn_off --no_trim &&
python {script_file} --input_file {os.path.join(SSH_DATA, input_file)} -o {out_folder} --regions {identifier5  + '.npy'} --find_syn_off --no_trim &&
python {script_file} --input_file {os.path.join(SSH_DATA, input_file)} -o {out_folder} --regions {identifier6  + '.npy'} --find_syn_off --no_trim &&
python {script_file} --input_file {os.path.join(SSH_DATA, input_file)} -o {out_folder} --regions {identifier7  + '.npy'} --find_syn_off --no_trim &&
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

print('scp %s/%s %s. &&' % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s. &&' % (SCRIPT_DIR, data_module, SSH_HOME))
print('scp %s/%s %s. &&' % (SARS_DIR, identifier + '.npy', SSH_HOME))
print('scp %s/%s %s. &&' % (SARS_DIR, identifier2 + '.npy', SSH_HOME))
print('scp %s/%s %s. &&' % (SARS_DIR, identifier3 + '.npy', SSH_HOME))
print('scp %s/%s %s. &&' % (SARS_DIR, identifier4 + '.npy', SSH_HOME))
print('scp %s/%s %s. &&' % (SARS_DIR, identifier5 + '.npy', SSH_HOME))
print('scp %s/%s %s. &&' % (SARS_DIR, identifier6 + '.npy', SSH_HOME))
print('scp %s/%s %s. &&' % (SARS_DIR, identifier7 + '.npy', SSH_HOME))
print('scp %s/%s %s.'    % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

print('sbatch %s' % job_file)
print('')

print('scp -r %s%s %s &&' % (SSH_HOME, out_folder, SARS_DIR))
print('scp -r %s%s %s' % (SSH_HOME, REF_TAG + '.fasta', SARS_DIR))

scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/processing-multiallele.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/regions-2021-08-14.npy blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/regions-2021-08-14b.npy blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-processing.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-processing.sh

scp -r blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14 /Users/brianlee/SARS-CoV-2-Data &&
scp -r blee098@cluster.hpcc.ucr.edu:EPI_ISL_402125.fasta /Users/brianlee/SARS-CoV-2-Data


### Finding time series with good sampling in each region

In [79]:
# Finding time series with good sampling

input_dir     = os.path.join(SSH_DATA, 'data')
output_dir    = input_dir
script_file   = 'trim-sampling-intervals.py'

job_file      = 'job-trim-intervals.sh'
job_str       = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=300G
#SBATCH --time=20:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/sampling-error
#SBATCH -o ./MPL/out/sampling-out

module purge
module load anaconda3

python {script_file} --input_dir {input_dir} -o {output_dir}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()


print('scp %s/%s %s. &&'   % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

print('sbatch %s' % job_file)
print('')

print('scp -r %s%s %s' % (SSH_HOME, output_dir, DATA_DIR))

scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/trim-sampling-intervals.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-trim-intervals.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-trim-intervals.sh

scp -r blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14 /Users/brianlee/SARS-CoV-2-Data/2021-08-14


### Eliminating sites that never rise to a very high frequency or have a mutation only in a few sequences in a region

In [426]:
# Finding maximum frequencies in each region

input_dir     = os.path.join(SSH_DATA, 'data', 'genome-trimmed')
output_dir    = os.path.join(SSH_DATA, 'data', 'max-freqs')
script_file   = 'find-max-frequencies.py'

# run on cluster to find number of files
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
print('')

num_files     = 161    # the number of data files 

job_file      = 'job-max-freqs.sh'
job_str       = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=200G
#SBATCH --time=2-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freqs-error-%a
#SBATCH -o ./MPL/out/freqs-out-%a
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)

module purge
module load anaconda3

file=${{files[$SLURM_ARRAY_TASK_ID]}}
python {script_file} --input_file \"$file\" -o {output_dir}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()


print('scp %s/%s %s. &&'   % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

print('sbatch %s' % job_file)

ls /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/genome-trimmed | wc -l

scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/find-max-frequencies.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-max-freqs.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-max-freqs.sh


In [11]:
# Combining maximum frequencies in different regions

input_dir     = os.path.join(SSH_DATA, 'data', 'max-freqs')
out_dir       = os.path.join(SSH_DATA, 'data')
script_file   = 'combine-max-freqs.py'

job_file      = 'job-combine-max-freqs.sh'
job_str       = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=200G
#SBATCH --time=20:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freqs-combine-error
#SBATCH -o ./MPL/out/freqs-combine-out

module purge
module load anaconda3

python {script_file} --input_dir {input_dir} -o {out_dir}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()


print('scp %s/%s %s. &&'   % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

print('sbatch %s' % job_file)

scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/combine-max-freqs.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-combine-max-freqs.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-combine-max-freqs.sh


In [13]:
# Filtering sites

input_dir     = os.path.join(SSH_DATA, 'data', 'genome-trimmed')
out_dir       = os.path.join(SSH_DATA, 'data')
freq_file     = os.path.join(SSH_DATA, 'data', 'maximum-frequencies.npz')
script_file   = 'filter-sites.py'

job_file      = 'job-filter-sites.sh'
job_str       = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=200G
#SBATCH --time=20:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/filter-error
#SBATCH -o ./MPL/out/filter-out

module purge
module load anaconda3

python {script_file} --input_dir {input_dir} -o {out_dir} --max_file {freq_file} --max_freq 0.05 && \
python {script_file} --input_dir {input_dir} -o {out_dir} --max_file {freq_file} --max_freq 0.01 && \
python {script_file} --input_dir {input_dir} -o {out_dir} --max_file {freq_file} --max_freq 0.1
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()


print('scp %s/%s %s. &&'   % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

print('sbatch %s' % job_file)
print('')
print(f'scp -r {SSH_HOME}/{SSH_DATA}/data {SARS_DIR}')

scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/filter-sites.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-filter-sites.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-filter-sites.sh


### Counting number of sequences used in final analysis

In [476]:
counts = 0
for file in os.listdir(os.path.join(DATA_DIR, 'freq_0.05')):
    filepath = os.path.join(DATA_DIR, 'freq_0.05', file)
    nVec     = np.load(filepath, allow_pickle=True)['nVec']
    counts  += np.sum([np.sum(nVec[t]) for t in range(len(nVec))])
print(f'total number of sequences used = {counts}')

total number of sequences used = 1652770.0


<a id='link'></a>
# Finding linked sites and classifying mutations as synonymous or nonsynonymous

### Finding linked sites on cluster

In [53]:
# Finding single and double site counts in each region

input_dir     = os.path.join(SSH_DATA, 'data', 'freq_0.01')
count_script  = 'epi-covar-parallel.py'
link_script   = 'epi-find-linked-parallel.py'
archive_loc   = os.path.join(SCRIPT_DIR, 'Archive-alt-vector')
out_dir       = os.path.join(SSH_DATA, 'data')
temp_dir      = os.path.join(SSH_DATA, 'data', 'counts')

# run on cluster to find number of files
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
num_files = 161    # the number of data files 

job_file1 = 'job-find-counts.sh'
job_str1  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=500G
#SBATCH --time=2-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/counts-error-%a
#SBATCH -o ./MPL/out/counts-out-%a
#SBATCH -p highmem
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)

module purge
module load anaconda3

cd ./Archive-alt-vector
g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11
pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python ../{count_script} --data \"$file\" -o $tempout --find_counts --pop_size 10000 -k 0.1 -R 2 --scratch /scratch/counts --timed 1
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()

# combining single and double site counts from the different regions to find linked sites

job_file2 = 'job-find-linked.sh'
job_str2  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=250G
#SBATCH --time=5-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/linked-error
#SBATCH -o ./MPL/out/linked-out

module purge
module load anaconda3

python {link_script} --data {temp_dir} --out_dir {out_dir} --timed 1 --multisite -q 5 --link_tol 0.8 --findLinkedRegional
""" 

f = open(os.path.join(SCRIPT_DIR, job_file2), 'w')
f.write('%s\n' % job_str2)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, count_script, SSH_HOME))
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, link_script, SSH_HOME))
print('scp -r %s %s. &&'   % (archive_loc, SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file1, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file2, SSH_HOME))
print('')

# run on cluster
print('sbatch %s' % job_file1)
print('sbatch %s' % job_file2)
print('')

# transfer files from cluster
print('scp %s%s/linked-sites.npy %s &&'               % (SSH_HOME, out_dir, DATA_DIR))
print('scp %s%s/linked-sites-regional.npy %s &&'      % (SSH_HOME, out_dir, DATA_DIR))
print('scp %s%s/nucleotide-counts.npz %s' % (SSH_HOME, out_dir, DATA_DIR))

ls /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/freq_0.05 | wc -l
scp /Users/brianlee/Python/MPL/epi-covar-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/Python/MPL/epi-find-linked-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp -r /Users/brianlee/Python/MPL/6-29-20-epidemiological/Archive-alt2 blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-find-counts.sh blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-find-linked.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-find-counts.sh
sbatch job-find-linked.sh

scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/linked-sites.npy /Users/brianlee/SARS-CoV-2-Data/2021-08-14 &&
scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/linked-sites-alleles.npy /Users/brianlee/SARS-

In [239]:
epsilon        = ['NSP13-260-0-T', 'S-13-1-T', 'S-152-2-T', 'NC-28271-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 
                  'S-452-1-G', 'N-205-1-T', 'ORF3a-57-2-T', 'NSP9-65-0-G'] # Done

alpha          = ['NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--',
                  'NSP6-108-1--', 'NSP6-108-2--', 'S-501-0-T', 'NSP12-412-2-T', 'NSP2-36-2-T', 'NSP3-183-1-T', 'NSP3-890-1-A', 
                  'NSP3-1089-2-T', 'NSP3-1412-1-C', 'NSP12-613-2-T', 'NSP12-912-2-C', 'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 
                  'S-69-2--', 'S-70-0--', 'S-143-2--', 'S-144-0--', 'S-144-1--', 'S-570-1-A', 'S-681-1-A', 'S-716-1-T', 'S-982-0-G', 
                  'S-1118-0-C', 'ORF8-27-0-T', 'ORF8-52-1-T', 'ORF8-73-1-G', 'N-3-0-C', 'N-3-1-T', 'N-3-2-A', 'N-235-1-T',
                  'N-203-1-A', 'N-203-2-A', 'N-204-0-C', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G']    # Done

gamma          = ['NSP9-31-2-T', 'NSP1-156-2-C', 'NSP3-10-2-T', 'NSP3-370-1-T', 'NSP3-977-0-C', 'NSP3-1200-2-G', 'NSP3-1298-2-G', 
                  'NSP12-140-2-T', 'NSP13-341-2-T', 'S-20-1-A', 'S-417-1-C', 'S-1027-1-T', 'ORF3a-253-0-C', 'ORF8-92-0-A', 'N-80-1-G', 
                  'S-190-2-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-501-0-T', 'NSP6-106-0--', 'NSP6-106-1--', 
                  'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--', 'NSP6-108-1--', 'NSP6-108-2--',
                  'N-203-1-A', 'N-203-2-A', 'N-204-0-C', 'N-202-0-T', 'N-202-1-C', 'S-18-0-T', 'S-26-0-T', 'S-655-0-T',
                  'S-1176-0-T']    # Done

twentyE_EU1    = ['NSP16-199-2-C', 'NSP1-60-2-C', 'NSP3-1189-2-T', 'M-93-2-G', 'N-220-1-T', 'ORF10-30-0-T', 'S-222-1-T',
                  'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G']    # Done
# This is Pango lineage B.1.177

delta          = ['NSP12-671-0-A', 'NC-209-T', 'NSP13-77-1-T', 'S-19-1-G', 'S-156-1--', 'S-156-2--', 'S-157-0--', 'S-157-1--', 'S-157-2--', 
                  'S-158-0--', 'S-478-1-A', 'S-681-1-G', 'S-950-0-A', 'ORF3a-26-1-T', 'M-82-1-C', 'ORF7a-82-1-C', 'ORF7a-120-1-T', 
                  'ORF8-119-0--', 'ORF8-119-1--', 'ORF8-119-2--', 'ORF8-120-0--', 'ORF8-120-1--', 'ORF8-120-2--', 'N-63-1-G', 'N-203-1-T', 
                  'N-377-0-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-452-1-G', 'NC-28270--', 'NSP4-492-1-T']    # Done

beta           = ['S-80-1-C', 'NSP3-837-2-T', 'S-215-1-G', 'E-71-1-T', 'S-240-2--', 'S-241-0--', 'S-242-1--', 'S-242-2--', 'S-243-0--', 
                  'S-240-1--', 'S-241-1--', 'S-241-2--', 'S-242-0--', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-501-0-T',
                  'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--', 
                  'NSP6-108-1--', 'NSP6-108-2--', 'S-417-2-T', 'S-484-0-A', 'S-701-1-T', 'ORF3a-57-2-T', 'NSP2-85-1-T', 'NSP5-90-1-G',
                  'N-205-1-T', 'NC-173-T', 'ORF8-120-2-T']    # Done

lambda_new     = ['S-246-2--', 'NSP3-1569-0-G', 'S-247-0--', 'S-247-1--', 'S-247-2--', 'S-248-0--', 'S-248-1--', 'S-248-2--', 'S-249-0--', 
                  'S-249-1--', 'S-249-2--', 'S-250-0--', 'S-250-1--', 'S-250-2--', 'S-251-0--', 'S-251-1--', 'S-251-2--', 'S-252-0--', 
                  'S-252-1--', 'S-252-2--', 'S-253-0--', 'N-214-0-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 
                  'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--', 
                  'NSP6-108-1--', 'NSP6-108-2--', 'S-75-1-T', 'S-76-1-T', 'S-452-1-A', 'S-490-1-C', 'S-859-1-A', 'NSP3-428-1-T',
                  'NSP3-1469-0-T', 'NSP4-438-1-C', 'NSP4-492-1-T', 'NSP5-15-0-A'] # Done

iota           = ['N-234-2-A', 'S-5-0-T', 'S-95-1-T', 'S-253-1-G', 'S-484-0-A', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G',
                 'S-701-1-T', 'NSP13-88-2-C', 'ORF3a-42-1-T', 'ORF3a-67-2-T', 'NSP2-85-1-T', 'NSP4-438-1-C', 'NSP6-106-0--', 'NSP6-106-1--', 
                 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--',  'NSP6-108-1--', 'NSP6-108-2--', 'N-199-1-T',
                 'N-232-2-A', 'ORF8-11-1-T', 'NSP15-214-2-G', 'NC-28270--']

D614G          = ['NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G']    # Done

B1_1_318       = ['NSP15-320-0-A', 'S-575-2-C', 'S-1238-2-A', 'ORF7b-44-1--', 'NC-27887--', 'NC-27888--', 'NC-27889--', 'NC-27890--', 
                  'NC-27891--', 'NC-27892--', 'ORF8-1-0--', 'ORF8-1-1--', 'ORF8-1-2--', 'NSP4-173-1-T', 'S-796-0-C', 'ORF8-2-0--', 
                  'ORF8-2-1--', 'ORF8-2-2--', 'ORF8-3-0--', 'ORF8-3-1--', 'NC-28270-G', 'N-208-2--', 'N-209-0--', 'N-208-1--',
                  'NSP3-378-1-T', 'NSP3-1693-2-T', 'NSP5-21-1-T', 'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 
                  'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--', 'NSP6-108-1--', 'NSP6-108-2--', 'S-95-1-T', 'S-143-2--', 'S-144-0--', 
                  'S-144-1--', 'S-484-0-A', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-681-1-A', 'M-82-1-C',
                  'N-203-1-A', 'N-203-2-A', 'N-204-0-C'] # Add N-234-2-T, Many others

#all_variants   = [epsilon, alpha, beta, gamma, lambda_new, delta, twentyE_EU1, iota, D614G]
all_variants   = [epsilon, alpha, beta, gamma, lambda_new, delta, twentyE_EU1, D614G]
#all_variants    = [epsilon, alpha, beta, gamma, lambda_new, delta, twentyE_EU1, iota, D614G, B1_1_318]
np.save(os.path.join(SARS_DIR, 'variants.npy'), all_variants)

var_dic = {'alpha' : alpha, 'epsilon' : epsilon, 'beta' : beta, 'gamma' : gamma, 'lambda' : lambda_new, 'delta' : delta, '20E\(EU1\)' : twentyE_EU1, 'B.1' : D614G}

### Classifying mutations as synonymous or nonsynonymous on the cluster

In [50]:
script_file   = 'find-nonsynonymous.py'
out_dir       = os.path.join(SSH_DATA, 'data')

job_file      = 'job-find-nonsynonymous.sh'
job_str       = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=300G
#SBATCH --time=1-0
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/error-synonymous
#SBATCH -o ./MPL/out/out-synonymous

module purge
module load anaconda3

python {script_file} --data {out_dir} --multisite
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

print('scp %s/%s %s. &&'    % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s. &&'    % (SCRIPT_DIR, 'data_processing.py', SSH_HOME))
#print('scp %s/%s %s/%s &&'  % (SARS_DIR, 'EPI_ISL_402125.fasta', SSH_HOME, SSH_DATA))
print('scp %s/%s %s.'       % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

print('sbatch %s' % job_file)
print('')

print('scp %s%s/%s %s &&' % (SSH_HOME, out_dir, 'synonymous.npz', DATA_DIR))
print('scp %s%s/%s %s &&' % (SSH_HOME, out_dir, 'synonymous-prot.npz', DATA_DIR))
print('scp %s%s/%s %s'    % (SSH_HOME, out_dir, 'protein-changes', DATA_DIR))
print(f'scp {SSH_HOME}/{SSH_DATA}/{REF_TAG}.fasta {SARS_DIR}')

scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/find-nonsynonymous.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-find-nonsynonymous.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-find-nonsynonymous.sh

scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/synonymous.npz /Users/brianlee/SARS-CoV-2-Data/2021-08-14 &&
scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/synonymous-prot.npz /Users/brianlee/SARS-CoV-2-Data/2021-08-14 &&
scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/protein-changes /Users/brianlee/SARS-CoV-2-Data/2021-08-14


In [ ]:
# Changing the amino acid for nucleotide mutations that are linked to other nucleotide mutations in the same codon

# RENAME protein-changes-test and synonymous-test IN SUBSEQUENT CELLS AND THEN DELETE THE CREATION OF THESE FILES

dp.fix_aa_changes(os.path.join(DATA_DIR, 'synonymous-prot.npz'),
                  os.path.join(DATA_DIR, 'linked-sites.npy'),
                  os.path.join(DATA_DIR, 'synonymous-prot-test'))

data = np.load(os.path.join(DATA_DIR, 'synonymous-prot-test.npz'), allow_pickle=True)
types = data['types']
nucs = data['nuc_index']
locs = data['locations']
aa_changes = data['aa_changes']

prot_out_file = os.path.join(DATA_DIR, 'protein-changes-test')
g = open(prot_out_file, 'w')
for i in range(len(locs)):
    g.write('%s,%s\n' % (locs[i], aa_changes[i]))
g.close()

prot_out_file = os.path.join(DATA_DIR, 'protein-changes')
g = open(prot_out_file, 'w')
for i in range(len(locs)):
    g.write('%s,%s\n' % (locs[i], aa_changes[i]))
g.close()
    
syn_out_file = os.path.join(DATA_DIR, 'synonymous-test.npz')
np.savez_compressed(syn_out_file, locations=locs, types=types)
syn_out_file = os.path.join(DATA_DIR, 'synonymous.npz')
np.savez_compressed(syn_out_file, locations=locs, types=types)

<a id='infer'></a>
# Inferring selection coefficients

### Inferring the selection coefficients for all of the sites using all of the regions

In [139]:
# infering the selection coefficients using a cutoff frequency of 5%
input_dir      = os.path.join(SSH_DATA, 'data', 'freq_0.05')
freq_script    = 'epi-covar-parallel.py'
inf_script     = 'epi-inf-parallel.py'
archive_loc    = os.path.join(SCRIPT_DIR, 'Archive-vector')
temp_dir       = os.path.join(SSH_DATA, 'data', 'freqs')
out_file       = os.path.join(SSH_DATA, f'infer-{DATA_DATE}')

# run on cluster
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
print('')

num_files     = 161    # the number of data files that must be read from the directory

job_file1     = 'job-find-freqs-multisite.sh'
job_str1      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=250G
#SBATCH --time=4-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq-error-%a
#SBATCH -o ./MPL/out/freq-out-%a
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)
module purge
module load anaconda3


cd ./Archive-vector
g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11
pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python ../{freq_script} --data \"$file\" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch /scratch/freqs --timed 1
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()

regularization=[1, 5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 100, 125, 150, 175, 200]

job_file2 = 'job-infer-multisite.sh'
job_str2  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=400G
#SBATCH --time=20:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/inf-error-%a
#SBATCH -o ./MPL/out/inf-out-%a
#SBATCH -p highmem
#SBATCH --array=0-16

regularization=(1 5 10 15 20 25 30 40 50 60 70 80 100 125 150 175 200)
module purge
module load anaconda3

outfile={out_file}-g-${{regularization[$SLURM_ARRAY_TASK_ID]}}
python {inf_script} --data {temp_dir} --timed 1 -o $outfile -q 5 --g1 ${{regularization[$SLURM_ARRAY_TASK_ID]}}
"""

f = open(os.path.join(SCRIPT_DIR, job_file2), 'w')
f.write('%s\n' % job_str2)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, freq_script, SSH_HOME))
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, inf_script, SSH_HOME))
print('scp -r %s %s. &&'   % (archive_loc, SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file1, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file2, SSH_HOME))
print('')

# run on cluster
print('sbatch %s &&' % job_file1)
print('sbatch %s &&' % job_file2)
print('mkdir %s &&'  % os.path.join(SSH_DATA, f'inf-{DATA_DATE}'))
print('mv %s %s'     % (os.path.join(SSH_DATA, f'infer-{DATA_DATE}-*'), os.path.join(SSH_DATA, f'inf-{DATA_DATE}')))
print('')

# transfer files from cluster
print('scp -r %s%s/%s %s' % (SSH_HOME, SSH_DATA, f'inf-{DATA_DATE}', INF_DIR))

ls /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/freq_0.05 | wc -l

scp /Users/brianlee/Python/MPL/epi-covar-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/Python/MPL/epi-inf-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp -r /Users/brianlee/Python/MPL/6-29-20-epidemiological/Archive blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-find-freqs-multisite.sh blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-infer-multisite.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-find-freqs-multisite.sh &&
sbatch job-infer-multisite.sh &&
mkdir /rhome/blee098/bigdata/SARS-CoV-2-Data/inf-2021-08-14 &&
mv /rhome/blee098/bigdata/SARS-CoV-2-Data/infer-2021-08-14-* /rhome/blee098/bigdata/SARS-CoV-2-Data/inf-2021-08-14

scp -r blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/in

### Eliminating mutations that are not observed and reference nucleotides

In [34]:
cutoff    = 0
regs      = [20, 30, 40, 50, 60]
if not os.path.exists(os.path.join(DATA_DIR, 'sensitivity-data')):
    os.mkdir(os.path.join(DATA_DIR, 'sensitivity-data'))
for reg in regs:

    inf_data  = np.load(os.path.join(INF_DIR, f'inf-{DATA_DATE}', f'infer-{DATA_DATE}-g-{reg}.npz'), allow_pickle=True)
    alleles   = inf_data['allele_number']
    selection = inf_data['selection']
    errors    = inf_data['error_bars']
    s_ind     = inf_data['selection_independent']
    numerator = inf_data['numerator']
    covar_int = inf_data['covar_int']
    locations = inf_data['locations']
    times     = inf_data['times']
    
    data      = np.load(os.path.join(DATA_DIR, 'nucleotide-counts.npz'), allow_pickle=True)
    muts      = data['allele_number']
    counts    = data['counts']
    mask      = np.isin(muts, alleles)
    counts    = counts[mask]

    #print('data loaded')
    clip_start= 150
    nucs      = np.array([i[:-2] for i in alleles], dtype=int)

    mask      = np.nonzero(np.logical_and(np.logical_and(counts>cutoff, selection!=0), nucs > clip_start))[0]

    selection = selection[mask]
    alleles   = alleles[mask]
    errors    = errors[mask]
    s_ind     = s_ind[mask]
    numerator = numerator[mask]
    covar_int = covar_int[mask][:, mask]

    #print('selection data filtered')
    
    np.savez_compressed(os.path.join(DATA_DIR, 'sensitivity-data', f'infer-{DATA_DATE}-g-{reg}-observed.npz'), covar_int=covar_int, 
                       selection=selection, selection_independent=s_ind, error_bars=errors, numerator=numerator,
                       allele_number=alleles)
    
    dp.website_file2(os.path.join(DATA_DIR, 'sensitivity-data', f'infer-{DATA_DATE}-g-{reg}-observed.npz'),
                     os.path.join(DATA_DIR, 'synonymous-prot-test.npz'),
                     os.path.join(DATA_DIR, 'linked-sites.npy'), 
                     os.path.join(DATA_DIR, f'selection-g-40-observed.csv'))

data loaded
selection data filtered


### Eliminate noncoding sites and rerun inference

In [277]:
# Eliminate all noncoding sites

data      = np.load(os.path.join(DATA_DIR, 'nucleotide-counts.npz'), allow_pickle=True)
muts      = data['allele_number']
counts    = data['counts']

inf_data  = np.load(os.path.join(INF_DIR, f'inf-{DATA_DATE}', f'infer-{DATA_DATE}-g-40.npz'), allow_pickle=True)
alleles   = inf_data['allele_number']
selection = inf_data['selection']
errors    = inf_data['error_bars']
s_ind     = inf_data['selection_independent']
numerator = inf_data['numerator']
covar_int = inf_data['covar_int']
locations = inf_data['locations']
proteins  = [dp.get_label2(i).split('-')[0] for i in alleles]

mask      = np.isin(muts, alleles)
muts      = muts[mask]
counts    = counts[mask]

mask      = np.nonzero(np.array(proteins)!='NC')[0]
selection = selection[mask]
alleles   = alleles[mask]
errors    = errors[mask]
s_ind     = s_ind[mask]
numerator = numerator[mask]
covar_int = covar_int[mask][:, mask]
counts    = counts[mask]

q   = 5
N   = 10000
k   = 0.1
g   = 40
R   = 2
g1  = np.array(g, dtype=float) * (N * k * R) / (R + k)
for i in range(len(covar_int)):
    covar_int[i, i] += g1

# infer selection coefficients
s      = linalg.solve(covar_int, numerator, assume_a='sym')
s_ind  = numerator / np.diag(covar_int)
errors = 1 / np.sqrt(np.absolute(np.diag(covar_int)))

# normalize reference nucleotide to zero
L = int(len(s) / 5)
selection  = np.reshape(s, (L, q))
selection_nocovar = np.reshape(s_ind, (L, q))
ref_seq, ref_tag  = dp.get_MSA(REF_TAG +'.fasta')
ref_seq  = list(ref_seq[0])
allele_number = np.unique([int(i[:-2]) for i in alleles])
ref_poly = np.array(ref_seq)[allele_number]

s_new = []
s_SL  = []
for i in range(L):
    idx = NUC.index(ref_poly[i])
    temp_s    = selection[i]
    temp_s    = temp_s - temp_s[idx]
    temp_s_SL = selection_nocovar[i]
    temp_s_SL = temp_s_SL - temp_s_SL[idx]
    s_new.append(temp_s)
    s_SL.append(temp_s_SL)
selection         = s_new
selection_nocovar = s_SL

selection         = np.array(selection).flatten()
selection_nocovar = np.array(selection_nocovar).flatten()
error_bars        = errors


mask      = np.nonzero(np.logical_and(counts>0, selection!=0))[0]
selection = selection[mask]
alleles   = alleles[mask]
errors    = errors[mask]
s_ind     = s_ind[mask]
numerator = numerator[mask]
covar_int = covar_int[mask][:, mask]

print('selection data filtered')
    
np.savez_compressed(os.path.join(INF_DIR, f'infer-{DATA_DATE}-g-40-observed-coding.npz'), covar_int=covar_int, 
                   selection=selection, selection_independent=s_ind, error_bars=errors, numerator=numerator,
                   allele_number=alleles)

In [242]:
dp.website_file2(os.path.join(INF_DIR, f'infer-{DATA_DATE}-g-40-observed-coding.npz'),
                 os.path.join(DATA_DIR, 'synonymous-prot.npz'),
                 os.path.join(DATA_DIR, 'linked-sites.npy'),
                 os.path.join(DATA_DIR, f'selection-g-40-observed-coding.csv'))

23164
23164


### infering the selection coefficients using a cutoff frequency of 1%

In [277]:
# infering the selection coefficients using a cutoff frequency of 1%
input_dir     = os.path.join(SSH_DATA, 'data', 'freq_0.01')
freq_script   = 'epi-covar-parallel.py'
inf_script    = 'epi-inf-parallel.py'
archive_loc   = os.path.join(SCRIPT_DIR, 'Archive-vector')
temp_dir      = os.path.join(SSH_DATA, 'data', 'freqs-0.01')
out_file      = os.path.join(SSH_DATA, f'infer-{DATA_DATE}-1pct')

# run on cluster
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
print('')

num_files     = 161    # the number of data files that must be read from the directory

job_file3     = 'job-freqs2.sh'
job_str3      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=400G
#SBATCH --time=3-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq2-error-%a
#SBATCH -o ./MPL/out/freq2-out-%a
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)

module purge
module load anaconda3

cd ./Archive-vector
g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11
pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python ../{freq_script} --data \"$file\" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {SCRATCH}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file3), 'w')
f.write('%s\n' % job_str3)
f.close()

job_file4 = 'job-infer2.sh'
job_str4  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=250G
#SBATCH --time=5-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/inf2-error
#SBATCH -o ./MPL/out/inf2-out

module purge
module load anaconda3

python {inf_script} --data {temp_dir} --timed 1 -o {out_file} -q 5 --g1 40 --eliminateNC
"""

f = open(os.path.join(SCRIPT_DIR, job_file4), 'w')
f.write('%s\n' % job_str4)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, freq_script, SSH_HOME))
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, inf_script, SSH_HOME))
print('scp -r %s %s. &&'   % (archive_loc, SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file3, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file4, SSH_HOME))
print('')

# run on cluster
print('sbatch %s &&' % job_file3)
print('sbatch %s' % job_file4)
print('')

# transfer files from cluster
print('scp %s%s %s'      % (SSH_HOME, out_file + '.npz', INF_DIR))

ls /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/freq_0.01 | wc -l

scp /Users/brianlee/Python/MPL/epi-covar-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/Python/MPL/epi-inf-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp -r /Users/brianlee/Python/MPL/6-29-20-epidemiological/Archive blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-find-freqs2.sh blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-infer-multisite2.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-find-freqs2.sh &&
sbatch job-infer-multisite2.sh

scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/infer-2021-08-14-cutoff0.01.npz /Users/brianlee/Python/MPL/6-29-20-epidemiological


In [383]:
# Finding frequency trajectories
input_dir     = os.path.join(SSH_DATA, 'data', 'freq_0.01')
freq_script   = 'epi-traj-parallel.py'
temp_dir      = os.path.join(SSH_DATA, 'data', 'trajectories-1pct')

# run on cluster
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
print('')

num_files     = 161    # the number of data files that must be read from the directory

job_file1     = 'job-find-traj.sh'
job_str1      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=150G
#SBATCH --time=5-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq-uk-error-%a
#SBATCH -o ./MPL/out/freq-uk-out-%a
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)
module purge
module load anaconda3

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python {freq_script} --data \"$file\" -o $tempout -q 5 --timed 1 --window 10
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()


# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, freq_script, SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file1, SSH_HOME))
print('')

# run on cluster
print('sbatch %s &&' % job_file1)
print('')

# transfer files from cluster
print(f'scp -r {SSH_HOME}{temp_dir} {DATA_DIR}')

ls /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/freq_0.01 | wc -l

scp /Users/brianlee/Python/MPL/epi-traj-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-find-traj.sh blee098@cluster.hpcc.ucr.edu:. &&

sbatch job-find-traj.sh &&

scp -r blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/trajectories-1pct /Users/brianlee/SARS-CoV-2-Data/2021-08-14


In [ ]:
# Combine trajectory files
traj      = []
muts      = []
times     = []
locs      = []
t_full    = []
traj_full = []
traj_dir  = os.path.join(DATA_DIR, 'trajectories-1pct')
for file in os.listdir(traj_dir):
    data = np.load(os.path.join(traj_dir, file))
    times.append(data['times'])
    muts.append(data['allele_number'])
    traj.append(data['traj'])
    locs.append(file[:-4])

np.savez_compressed(os.path.join(DATA_DIR, 'trajectories-1pct.npz'),
                    times=times, mutant_sites=muts, traj=traj, locations=locs, 
                    times_nosmooth=t_full, traj_nosmooth=traj_full)

In [18]:
# Find trajectories for 20E-EU1
os.chdir(SCRIPT_DIR)
import data_processing as dp
reload(dp)
os.chdir(SARS_DIR)
times    = []
traj     = []
data_dir = os.path.join(DATA_DIR, 'freq_0.05')
for file in os.listdir(data_dir):
    loc_split = file.split('-')
    if loc_split[0]=='europe' and loc_split[1]=='united kingdom' and loc_split[2]=='england_wales_scotland':
        filepath = os.path.join(data_dir, file)
        data     = np.load(filepath, allow_pickle=True)
        nVec     = data['nVec']
        sVec     = data['sVec']
        t_temp   = data['times']
        muts     = data['mutant_sites']
        muts     = [dp.get_label(i) for i in muts]
        traj_temp= dp.trajectory_calc_20e_eu1(nVec, sVec, muts, d=5)
        traj_temp= dp.moving_average(traj_temp, window=10)
        traj.append(traj_temp)
        times.append(t_temp[9:])

f = open(os.path.join(DATA_DIR, '20E-EU1-trajectory.csv'), mode='w')
f.write('time,frequency\n')
for i in range(len(times)):
    for j in range(len(times[i])):
        f.write(f'{times[i][j]},{traj[i][j]:0.8f}\n')
f.close()

In [113]:
# Finding number of sites in final inference
inf_data  = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-1pct.npz'), allow_pickle=True)
muts      = inf_data['allele_number']
print(f'number of mutations in is {4 * len(muts) / 5}')

number of mutations in is 79696.0


In [13]:
# Eliminating sites that are infrequently observed (frequency never about 1% in any region and total observations > 5)
reg       = 40
count_min = 5

inf_data  = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-1pct.npz'), allow_pickle=True)
alleles   = inf_data['allele_number']
selection = inf_data['selection']
errors    = inf_data['error_bars']
s_ind     = inf_data['selection_independent']
locations = inf_data['locations']
times     = inf_data['times']
clip_start= 150
nucs      = np.array([i[:-2] for i in alleles], dtype=int)

# Eliminating reference nucleotides and mutations that never rise above a frequency of 1%
traj_dir       = os.path.join(DATA_DIR, 'trajectories-1pct')
max_freq       = np.zeros(len(alleles))
alleles_sorted = np.argsort(alleles)

for file in os.listdir(traj_dir):
    data = np.load(os.path.join(traj_dir, file), allow_pickle=True)
    muts = data['allele_number']
    traj = data['traj']
    mask = np.isin(muts, alleles)
    muts = muts[mask]
    traj = traj[:, mask]
    maxs = np.amax(traj, axis=0)
    pos  = np.searchsorted(alleles[alleles_sorted], muts)
    pos  = alleles_sorted[pos]
    for j in range(len(maxs)):
        max_freq[pos[j]] = max(max_freq[pos[j]], maxs[j])
        
        
cutoff    = 0.01
mask      = np.nonzero(np.logical_and(max_freq>cutoff, selection!=0))[0]

selection = selection[mask]
alleles   = alleles[mask]
errors    = errors[mask]
s_ind     = s_ind[mask]

count_data = np.load(os.path.join(DATA_DIR, 'nucleotide-counts.npz'), allow_pickle=True)
counts     = count_data['counts']
alleles2   = count_data['allele_number']

mask1      = np.isin(alleles2, alleles)
alleles2   = alleles2[mask1]
counts     = counts[mask1]

mask2      = np.isin(alleles, alleles2)
alleles    = alleles[mask2]
selection  = selection[mask2]
s_ind      = s_ind[mask2]
errors     = errors[mask2]

mask3      = np.nonzero(counts>count_min)[0]
alleles    = alleles[mask3]
selection  = selection[mask3]
s_ind      = s_ind[mask3]
errors     = errors[mask3]

traj_data = np.load(os.path.join(DATA_DIR, 'trajectories-1pct.npz'), allow_pickle=True)
times     = traj_data['times']
muts      = traj_data['mutant_sites']
locations = traj_data['locations']
traj      = traj_data['traj']
    
np.savez_compressed(os.path.join(DATA_DIR, 'sensitivity-data', f'infer-{DATA_DATE}-g-{reg}-1pct-observed.npz'),
                    selection=selection, selection_independent=s_ind, error_bars=errors,
                    allele_number=alleles, traj=traj, mutant_sites=muts, locations=locations,
                    times=times)

#np.savez_compressed(os.path.join(INF_DIR, f'infer-{DATA_DATE}-g-{reg}-1pct-observed.npz'),
#                    selection=selection, selection_independent=s_ind, error_bars=errors,
#                    allele_number=alleles, traj=traj, mutant_sites=muts, locations=locations,
#                    times=times)
    
dp.website_file2(os.path.join(DATA_DIR, 'sensitivity-data', f'infer-{DATA_DATE}-g-{reg}-1pct-observed.npz'),
                 os.path.join(DATA_DIR, 'synonymous-prot-test.npz'),
                 os.path.join(DATA_DIR, 'linked-sites.npy'), 
                 os.path.join(DATA_DIR, f'selection-g-40-1pct-observed.csv'))

In [115]:
# Number of observed amino acid mutations
data = np.load(os.path.join(DATA_DIR, 'sensitivity-data', f'infer-{DATA_DATE}-g-40-1pct-observed.npz'), allow_pickle=True)
muts = data['allele_number']
print(f'number of observed mutations is {len(muts)}')

number of observed mutations is 21050


__Finding coefficients for variants__

In [4]:
dp.linked_analysis_alt(os.path.join(DATA_DIR, 'sensitivity-data', f'infer-{DATA_DATE}-g-40-1pct-observed.npz'),
                       os.path.join(DATA_DIR, 'linked-sites.npy'), 
                       os.path.join(DATA_DIR, 'synonymous-prot-test.npz'),
                       os.path.join(SARS_DIR, 'variants.npy'),
                       os.path.join(DATA_DIR, 'linked-coefficients-g-40-1pct.csv'))

# Make a csv file containing the frequency trajectories for the different variants in different regions
dp.linked_traj_csv(os.path.join(DATA_DIR, 'linked-coefficients-g-40-1pct.csv'), 
                   os.path.join(DATA_DIR, 'trajectories-1pct.npz'), 
                   os.path.join(DATA_DIR, 'linked-trajectories-1pct'))

In [ ]:
# Change nucleotide mutations so they are indexed at 1 instead of 0
df = pd.read_csv(os.path.join(DATA_DIR, 'linked-coefficients-g-40-1pct.csv'), index_col=False)
nucs = list(df['nucleotides'])
nucs = [i.split('/') for i in nucs]
nucs_new = []
for i in nucs:
    nucs_temp = []
    for j in i:
        if j!='not_present':
            new_nuc = str(int(j) + 1)
        else:
            new_nuc = j
        nucs_temp.append(new_nuc)
    nucs_new.append(nucs_temp)
nucs = ['/'.join(i) for i in nucs_new]
df['nucleotides'] = nucs
df.to_csv(os.path.join(DATA_DIR, 'linked-coefficients-g-40-1pct.csv'), index=False)

### Nonsynonymous inference

In [ ]:
# Eliminating synonymous mutations
cutoff     = 0

count_data = np.load(os.path.join(DATA_DIR, 'nucleotide-counts.npz'), allow_pickle=True)
counts     = count_data['counts']
alleles2   = count_data['allele_number']

inf_data  = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-1pct.npz'), allow_pickle=True)
alleles   = inf_data['allele_number']
numerator = inf_data['numerator']
covar_int = inf_data['covar_int']

mask2     = np.isin(alleles, alleles2)
alleles   = alleles[mask2]
numerator = numerator[mask2]
covar_int = covar_int[mask2][:, mask2]

syn_data  = np.load(os.path.join(DATA_DIR, 'synonymous-prot-test.npz'))
syn_muts  = syn_data['nuc_index']
types     = syn_data['types']
types     = types[np.isin(syn_muts, alleles)]

idxs_keep = []
for i in range(int(len(alleles) / 5)):
    types_temp  = types[5 * i : 5 * (i + 1)]
    counts_temp = counts[5 * i : 5 * (i + 1)]
    types_temp  = [types_temp[j] for j in range(len(types_temp)) if counts_temp[j] > cutoff]
    if 'NS' in types_temp:
        for j in range(5):
            idxs_keep.append((5 * i) + j)
idxs_keep = np.array(idxs_keep)

numerator = numerator[idxs_keep]
covar_int = covar_int[idxs_keep][:, idxs_keep]
alleles   = alleles[idxs_keep]
counts    = counts[idxs_keep]

np.savez_compressed(os.path.join(INF_DIR, f'inf-data-nonsynonymous-{DATA_DATE}-1pct.npz'), allele_number=alleles, 
                    numerator=numerator, covar_int=covar_int, counts=counts)

In [ ]:
# Inferring the selection coefficients only for nonsynonymous sites 
inf_script    = 'epi-inf-nonsynonymous.py'
count_file    = 'nucleotide-counts.npz'
input_file    = f'infer-{DATA_DATE}-1pct.npz'
syn_file      = 'synonymous-prot-test.npz'
out_file      = os.path.join(SSH_DATA, f'infer-{DATA_DATE}-1pct-nonsyn-observed-v2')

job_file2 = 'job-infer-nonsyn.sh'
job_str2  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=400G
#SBATCH --time=2-00:00:00
#SBATCH -e ./MPL/out/inf-1pct-error
#SBATCH -o ./MPL/out/inf-1pct-out
#SBATCH -p batch

module purge
module load anaconda3


python {inf_script} --data {SSH_DATA}/{input_file} -k 0.1 -R 2 --pop_size 10000 --timed 1 --g1 40  -o {out_file} --syn_data {syn_file} --count_data {SSH_DATA}/{count_file}

"""

f = open(os.path.join(SCRIPT_DIR, job_file2), 'w')
f.write('%s\n' % job_str2)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, inf_script, SSH_HOME))
print(f'scp {INF_DIR}/{input_file} {SSH_HOME}{SSH_DATA} &&')
print(f'scp {DATA_DIR}/{count_file} {SSH_HOME}{SSH_DATA} &&')
print(f'scp {DATA_DIR}/{syn_file} {SSH_HOME} &&')
print(f'scp {os.path.join(SCRIPT_DIR, job_file2)} {SSH_HOME} ')
print('')
print(f'scp {SSH_HOME}{out_file}-unmasked.npz {INF_DIR}')

In [71]:
# Masking out mutations never above 1% frequency

inf_data  = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-1pct-nonsyn-observed-v2-unmasked.npz'), allow_pickle=True)
alleles   = inf_data['allele_number']
selection = inf_data['selection']
errors    = inf_data['error_bars']
s_ind     = inf_data['selection_independent']
mut_sites = [dp.get_label2(i) for i in alleles]

syn_data  = np.load(os.path.join(DATA_DIR, 'synonymous-prot-test.npz'))
syn_muts  = syn_data['nuc_index']
types     = syn_data['types']
types     = types[np.isin(syn_muts, alleles)]

# Finding maximum frequencies
traj_dir       = os.path.join(DATA_DIR, 'trajectories-1pct')
max_freq       = np.zeros(len(alleles))
alleles_sorted = np.argsort(alleles)
for file in os.listdir(traj_dir):
    data = np.load(os.path.join(traj_dir, file), allow_pickle=True)
    muts = data['allele_number']
    traj = data['traj']
    mask = np.isin(muts, alleles)
    muts = muts[mask]
    traj = traj[:, mask]
    maxs = np.amax(traj, axis=0)
    pos  = np.searchsorted(alleles[alleles_sorted], muts)
    pos  = alleles_sorted[pos]
    for j in range(len(maxs)):
        max_freq[pos[j]] = max(max_freq[pos[j]], maxs[j])
        
cutoff = 0.01

idxs_keep = np.logical_and(types=='NS', max_freq>cutoff)

alleles   = alleles[idxs_keep]
selection = selection[idxs_keep]
s_ind     = s_ind[idxs_keep]
errors    = errors[idxs_keep]

np.savez_compressed(os.path.join(INF_DIR, f'inf-data-nonsynonymous-{DATA_DATE}-1pct-test.npz'), allele_number=alleles, 
                    selection=selection, error_bars=errors, selection_independent=s_ind)

In [70]:
# Eliminate mutations that don't appear at least five times
data = np.load(os.path.join(INF_DIR, f'inf-data-nonsynonymous-{DATA_DATE}-1pct-test.npz'), allow_pickle=True)
muts = data['allele_number']
s    = data['selection']
err  = data['error_bars']
s_ind= data['selection_independent']

count_data = np.load(os.path.join(DATA_DIR, 'nucleotide-counts.npz'), allow_pickle=True)
counts     = count_data['counts']
alleles    = count_data['allele_number']

mask1      = np.isin(alleles, muts)
alleles    = alleles[mask1]
counts     = counts[mask1]

mask2      = np.isin(muts, alleles)
muts       = muts[mask2]
s          = s[mask2]
s_ind      = s_ind[mask2]
errors     = err[mask2]

mask3      = np.nonzero(counts>5)[0]
muts       = muts[mask3]
s          = s[mask3]
s_ind      = s_ind[mask3]
errors     = errors[mask3]
        
np.savez_compressed(os.path.join(INF_DIR, f'inf-data-nonsynonymous-{DATA_DATE}-1pct-cutoff.npz'), allele_number=muts, 
                    selection=s, error_bars=errors, selection_independent=s_ind)

dp.website_file2(os.path.join(INF_DIR, f'inf-data-nonsynonymous-{DATA_DATE}-1pct-cutoff.npz'),
                 os.path.join(DATA_DIR, 'synonymous-prot-test.npz'),
                 os.path.join(DATA_DIR, 'linked-sites.npy'), 
                 os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn.csv'))

13189
13189
13189
13189
13189
13189


In [ ]:
# Eliminate sites due to issue with alignment
bad_sites = ['28273-G', '28275-C', '28279-A', '28280--', '28281--']
df        = pd.read_csv(os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn.csv'), index_col=False)
nuc_nums  = list(df['nucleotide number'])
nucs      = list(df['nucleotide'])
labels    = [str(nuc_nums[i]) + '-' + nucs[i] for i in range(len(nucs))]
idxs_drop = [labels.index(i) for i in bad_sites]
df        = df.drop(index=idxs_drop)
df.to_csv(os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn.csv'), index=False)

In [415]:
# Change nucleotide indices to be indexed starting with 1 instead of 0 for publishing
df = pd.read_csv(os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn.csv'), index_col=False)
nucs = np.array(list(df['nucleotide number'])) + 1
df['nucleotide number'] = nucs
df.to_csv(os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn-paper.csv'), index=False)

In [339]:
# Find mutations that are in the same codon and strongly linked to one another and sum their selection coefficients
data  = np.load(os.path.join(DATA_DIR, 'linked-sites.npy'), allow_pickle=True)
nucs  = [np.array([data[i][j][-1] for j in range(len(data[i]))]) for i in range(len(data))]
idxs  = []
for i in range(len(data)):
    if len(np.array(nucs[i])[nucs[i]=='-']) != len(nucs[i]):
        idxs.append(i)
data = data[idxs]
#for i in data:
    #print(np.sort(i))
split = [[data[i][j].split('-') for j in range(len(data[i]))] for i in range(len(data))]
cods  = [[split[i][j][1] for j in range(len(split[i]))] for i in range(len(split))]
prots = [[split[i][j][0] for j in range(len(split[i]))] for i in range(len(split))]
    
#linked_same_codon = []
linked_groups     = []
for i in range(len(data)):
    group   = data[i]
    #g_nucs  = nucs[i]
    g_cods  = cods[i]
    g_prots = prots[i]
    prot_unique = np.unique(g_prots)
    for j in range(len(prot_unique)):
        temp_cods   = np.array(g_cods)[np.array(g_prots)==prot_unique[j]]
        temp_labs   = np.array(group)[np.array(g_prots)==prot_unique[j]]
        cods_unique = np.unique(temp_cods)
        for k in range(len(cods_unique)):
            labs_same_cod = temp_labs[temp_cods==cods_unique[k]]
            if len(labs_same_cod)>1:
                linked_groups.append(labs_same_cod)
                #for l in labs_same_cod:
                    #linked_same_codon.append(l)
#linked_same_codon = [i for i in linked_same_codon if i[-1]!='-']
linked_groups     = [[linked_groups[i][j] for j in range(len(linked_groups[i])) if linked_groups[i][j][-1]!='-'] for i in range(len(linked_groups))]
linked_groups     = [i for i in linked_groups if len(i)!=0]
linked_all        = [linked_groups[i][j] for i in range(len(linked_groups)) for j in range(len(linked_groups[i]))]


# collapse identical mutations and sum contributions
df_sel = pd.read_csv(os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn.csv'), memory_map=True)

f = open(os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn-collapsed.csv'), 'w')

headers = list(df_sel.columns)
f.write('%s\n' % ','.join(headers))

print('name\tn dupes\ttotal s')

dup_names = []
for iter, entry in df_sel.iterrows():
    if entry['protein']=='NC':
        continue
    loc      = str(int(entry['amino acid number in protein']))
    anc      = entry['amino acid mutation'][0]
    mut      = entry['amino acid mutation'][-1]
    var_name = str(entry['protein']) + '-' + anc + loc + mut
    label    = dp.get_label2(str(entry['nucleotide number']) + '-' + entry['nucleotide'])
    if var_name in dup_names:
        continue
    
    df_same = df_sel[(df_sel['protein']==entry['protein']) 
                     & (df_sel['amino acid number in protein']==entry['amino acid number in protein'])
                     & (df_sel['amino acid mutation']==entry['amino acid mutation'])]
    
    s = entry['selection coefficient']
    if len(df_same)>1:
        s = np.sum(df_same['selection coefficient'])
        if var_name[-1]!='-' and label not in linked_all:
            s = np.max(df_same['selection coefficient'])
        dup_names.append(var_name)
        if np.fabs(s)>0.03:
            print('%s\t%d\t%.3f' % (var_name, len(df_same), s))
        
    for i in range(len(headers)):
        if headers[i]=='selection coefficient':
            f.write('%s' % str(s))
        elif headers[i]=='linked sites':
            f.write('"%s"' % str(entry[headers[i]]))
#         elif headers[i]=='nucleotide number':
#             f.write('%s' % entry[headers[i]][:-2])
        else:
            f.write('%s' % str(entry[headers[i]]))
        if i!=len(headers)-1:
            f.write(',')
    f.write('\n')

f.close()

name	n dupes	total s
NSP6-S106-	3	0.054
NSP6-G107-	3	0.042
NSP6-F108-	3	0.031
S-L141-	3	0.031
S-G142-	3	0.036
ORF3a-V256-	2	0.035
N-D3-	4	0.057
N-S202-	3	0.035
N-M234I	3	0.031


In [259]:
# Change nucleotide indices to be indexed starting with 1 instead of 0 for publishing
df = pd.read_csv(os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn-collapsed.csv'), index_col=False)
nucs = np.array(list(df['nucleotide number'])) + 1
df['nucleotide number'] = nucs
df.to_csv(os.path.join(DATA_DIR, f'selection-g40-1pct-nonsyn-collapsed-paper.csv'), index=False)

### Inferring selection coefficients using a cutoff of 10%

In [105]:
# infering the selection coefficients using a cutoff frequency of 10% and for different values of the smoothing window
input_dir     = os.path.join(SSH_DATA, 'data', 'freq_0.1')
freq_script   = 'epi-covar-parallel.py'
inf_script    = 'epi-inf-parallel.py'
archive_loc   = os.path.join(SCRIPT_DIR, 'Archive-vector')
temp_dir      = os.path.join(SSH_DATA, 'data', 'freqs-0.1')
out_file      = os.path.join(SSH_DATA, f'infer-{DATA_DATE}-cutoff0.1')

temp_dir2      = os.path.join(SSH_DATA, 'data', 'freqs-0.1-window10')
out_file2      = os.path.join(SSH_DATA, f'infer-{DATA_DATE}-cutoff0.1-window10')
temp_dir3      = os.path.join(SSH_DATA, 'data', 'freqs-0.1-window20')
out_file3      = os.path.join(SSH_DATA, f'infer-{DATA_DATE}-cutoff0.1-window20')

# run on cluster
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
print('')

num_files     = 160    # the number of data files that must be read from the directory

job_file1     = 'job-find-freqs-0.1.sh'
job_str1      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=50G
#SBATCH --time=1-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq10-error-%a
#SBATCH -o ./MPL/out/freq10-out-%a
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)

module purge
module load anaconda3


cd ./Archive-vector
g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11
pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python ../{freq_script} --data \"$file\" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {SCRATCH}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()

job_file2     = 'job-find-freqs-window10.sh'
job_str2      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=50G
#SBATCH --time=1-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq10-error-%a
#SBATCH -o ./MPL/out/freq10-out-%a
#SBATCH -p batch
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)

module purge
module load anaconda3


cd ./Archive-vector
g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11
pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir2}
python ../{freq_script} --data \"$file\" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {SCRATCH} --window 10
""" 

f = open(os.path.join(SCRIPT_DIR, job_file2), 'w')
f.write('%s\n' % job_str2)
f.close()

job_file3     = 'job-find-freqs-window20.sh'
job_str3      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=50G
#SBATCH --time=1-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq10-error-%a
#SBATCH -o ./MPL/out/freq10-out-%a
#SBATCH -p batch
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)

module purge
module load anaconda3


cd ./Archive-vector
g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11
pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir3}
python ../{freq_script} --data \"$file\" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {SCRATCH} --window 20
""" 

f = open(os.path.join(SCRIPT_DIR, job_file3), 'w')
f.write('%s\n' % job_str3)
f.close()


# Inference
job_file4 = 'job-infer-0.1.sh'
job_str4  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --time=1-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/inf2-error
#SBATCH -o ./MPL/out/inf2-out

module purge
module load anaconda3

python {inf_script} --data {temp_dir}  --timed 1 -o {out_file}  -q 5 --g1 40 && \
python {inf_script} --data {temp_dir2} --timed 1 -o {out_file2} -q 5 --g1 40 && \
python {inf_script} --data {temp_dir3} --timed 1 -o {out_file3} -q 5 --g1 40
"""

f = open(os.path.join(SCRIPT_DIR, job_file4), 'w')
f.write('%s\n' % job_str4)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, freq_script, SSH_HOME))
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, inf_script, SSH_HOME))
print('scp -r %s %s. &&'   % (archive_loc, SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file1, SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file2, SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file3, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file4, SSH_HOME))
print('')

# run on cluster
print('sbatch %s &&' % job_file1)
print('sbatch %s &&' % job_file2)
print('sbatch %s &&' % job_file3)
print('sbatch %s' % job_file4)
print('')

# transfer files from cluster
print('scp %s%s %s &&' % (SSH_HOME, out_file  + '.npz', INF_DIR))
print('scp %s%s %s &&' % (SSH_HOME, out_file2 + '.npz', INF_DIR))
print('scp %s%s %s'    % (SSH_HOME, out_file3 + '.npz', INF_DIR))

ls /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/freq_0.1 | wc -l

scp /Users/brianlee/Python/MPL/epi-covar-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/Python/MPL/epi-inf-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp -r /Users/brianlee/Python/MPL/6-29-20-epidemiological/Archive blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-find-freqs-0.1.sh blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-infer-multisite-0.1.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-find-freqs-0.1.sh &&
sbatch job-infer-multisite-0.1.sh

scp blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/infer-2021-08-14-cutoff0.1.npz /Users/brianlee/Python/MPL/6-29-20-epidemiological


In [132]:
cutoff    = 0

data      = np.load(os.path.join(DATA_DIR, 'nucleotide-counts.npz'), allow_pickle=True)
muts      = data['allele_number']
counts    = data['counts']

regs      = [40]
for reg in regs:

    inf_data  = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-cutoff0.1.npz'), allow_pickle=True)
    alleles   = inf_data['allele_number']
    selection = inf_data['selection']
    errors    = inf_data['error_bars']
    s_ind     = inf_data['selection_independent']
    
    mask   = np.isin(muts, alleles)
    muts   = muts[mask]
    counts = counts[mask]
    
    #numerator = inf_data['numerator']
    #covar_int = inf_data['covar_int']
    locations = inf_data['locations']
    times     = inf_data['times']

    print('data loaded')
    clip_start= 150
    nucs      = np.array([i[:-2] for i in alleles], dtype=int)

    mask      = np.nonzero(np.logical_and(np.logical_and(counts>cutoff, selection!=0), nucs > clip_start))[0]

    selection = selection[mask]
    alleles   = alleles[mask]
    errors    = errors[mask]
    s_ind     = s_ind[mask]

    print('selection data filtered')
    np.savez_compressed(os.path.join(DATA_DIR, 'sensitivity-data', f'infer-{DATA_DATE}-g-{reg}-10pct-observed.npz'),
                       selection=selection, selection_independent=s_ind, error_bars=errors,
                       allele_number=alleles)
    
    dp.website_file2(os.path.join(DATA_DIR, 'sensitivity-data', f'infer-{DATA_DATE}-g-{reg}-10pct-observed.npz'),
                     os.path.join(DATA_DIR, 'synonymous-prot-test.npz'),
                     os.path.join(DATA_DIR, 'linked-sites.npy'), 
                     os.path.join(DATA_DIR, f'selection-g-40-1pct-observed.csv'))

data loaded
selection data filtered


In [ ]:
# make sampling directory
sample_dir = os.path.join(DATA_DIR, 'sampling-data')
input_dir  = os.path.join(DATA_DIR, 'genome-trimmed')
dp.make_sampling_dir(sample_dir, input_dir)

### Inferring the selection coefficients in the United Kingdom including migration

In [43]:
# Putting uk files in their own directory
uk_filepath = []
uk_filename = []
for file in sorted(os.listdir(os.path.join(DATA_DIR, 'freqs'))):
    file_split = file.split('-')
    if file_split[1] == 'united kingdom' and file_split[2] == 'england_wales_scotland':
        year_end  = int(file_split[-3])
        month_end = int(file_split[-2])
        day_end   = int(file_split[-1][:-4])  
        if (dt.date(year_end, month_end, day_end) - dt.date(2021, 5, 5)).days < 0:  # take only data before the middle of may
            uk_filepath.append(os.path.join(DATA_DIR, 'freqs', file))
            uk_filename.append(file)
uk_dir = os.path.join(DATA_DIR, 'genome-uk')
if not os.path.exists(uk_dir):
    os.mkdir(uk_dir)
for path in uk_filepath:
    shutil.copy(path, uk_dir)

print(f'scp -r {uk_dir} {SSH_HOME}/{SSH_DATA}/data')

scp /Users/brianlee/SARS-CoV-2-Data/2021-08-14/estimated-pop-size-uk.npy blee098@cluster.hpcc.ucr.edu://rhome/blee098/bigdata/SARS-CoV-2-Data
scp -r /Users/brianlee/SARS-CoV-2-Data/2021-08-14/genome-uk blee098@cluster.hpcc.ucr.edu://rhome/blee098/bigdata/SARS-CoV-2-Data


In [50]:
# Finding selection coefficients in the uk only (up to may 1st 2021)
out_file   = f'infer-{DATA_DATE}--2021-05-01-uk-g-10'
data       = os.path.join(SSH_DATA, 'data', 'genome-uk')
inf_script = 'epi-inf-parallel.py'

job_file  = 'job-infer-uk.sh'
job_str   = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=200G
#SBATCH --time=20:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/inf-error
#SBATCH -o ./MPL/out/inf-out

module purge
module load anaconda3

outfile={out_file}
python {inf_script} --data {data} --timed 1 -o $outfile -q 5 --g1 10
"""

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, inf_script, SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

# run on cluster
print('sbatch %s &&' % job_file)
print('')

print('scp %s%s %s' % (SSH_HOME, out_file + '.npz', INF_DIR))

scp /Users/brianlee/Python/MPL/epi-inf-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-infer-uk.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-infer-uk.sh &&

scp blee098@cluster.hpcc.ucr.edu:infer-2021-08-14--2021-05-01-uk-g-10.npz /Users/brianlee/Python/MPL/6-29-20-epidemiological


In [52]:
# sites linked to S-222 mutation
NUC = ['-', 'A', 'C', 'G', 'T']
infer_data = np.load(os.path.join(INF_DIR, out_file + '.npz'), allow_pickle=True)
allele_num = infer_data['allele_number']
labels     = [dp.get_label(i[:-2]) + '-' + i[-1] for i in allele_num]

freq_data  = np.load(os.path.join(DATA_DIR, 'freq_0.05', 
                     'europe-united kingdom-england_wales_scotland-2020-01-01-2021-01-01.n---2020-2-24-2021-1-1.npz'),
                     allow_pickle=True)
nVec       = freq_data['nVec']
sVec       = freq_data['sVec']
times      = freq_data['times']
mutants    = freq_data['mutant_sites']
mask       = np.nonzero(mutants<29700)[0]
#mask       = np.nonzero(np.logical_and(mutants<29700), mutants>150)[0]
mutants    = mutants[mask]
sVec_temp  = []
for t in sVec:
    sVec_t = []
    for i in t:
        sVec_t.append(i[mask])
    sVec_temp.append(sVec_t)
sVec = sVec_temp
nuc_nums   = [dp.get_label(i) for i in mutants]
loc_sites  = []
for site in mutants:
    for j in range(5):
        loc_sites.append(dp.get_label(site) + '-' + NUC[j])

link_sites = ['ORF10-30-0-T', 'N-220-1-T', 'M-93-2-G', 'S-222-1-T', 'NSP16-199-2-C', 'NSP3-1189-2-T', 'NSP1-60-2-C']    # The linked mutations that will inflow
sites      = [allele_num[labels.index(i)] for i in link_sites]

# Find a sequence containing the desired list of mutations
inflow_seq = None
seq_found  = False
for i in range(len(sVec)):
    for j in range(len(sVec[i])):
        muts = True
        for k in range(len(link_sites)):
            if sVec[i][j][nuc_nums.index(link_sites[k][:-2])] != NUC.index(link_sites[k][-1]):
                muts = False
        if muts:
            inflow_seq = sVec[i][j]
            seq_found  = True
            break
    if seq_found:
        break
print(inflow_seq)        

int64
[0 0 0 ... 4 4 1]


In [ ]:
# Extract the zipped file containing the estimated population size
est_dir = os.path.join(DATA_DIR, 'ihme-popsize-2021_09_01')
import zipfile
zipped = os.path.join(est_dir, 'reference_hospitalization_all_locs.csv.zip')
with zipfile.ZipFile(zipped, 'r') as zip_ref:
    zip_ref.extractall(est_dir)

In [2]:
# Running the inference with different numbers of infected individuals migrating into the United Kingdom
    
file1   = os.path.join(est_dir, 'reference_hospitalization_all_locs.csv')
data1   = pd.read_csv(file1)
popsize = list(data1.est_infections_mean)
dates   = list(data1.date)
locs    = [i.lower() for i in list(data1.location_name)]
uk_popsize  = np.array(popsize)[np.array(locs)=='united kingdom']
uk_dates    = np.array(dates)[np.array(locs)=='united kingdom']
uk_dates    = uk_dates[~np.isnan(uk_popsize)]
uk_times    = [(dt.date.fromisoformat(uk_dates[i]) - dt.date(2020,1,1)).days for i in range(len(uk_dates))]
uk_popsize  = uk_popsize[~np.isnan(uk_popsize)]

inf_data  = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}--2021-05-01-uk-g-10' + '.npz'), allow_pickle=True)
alleles   = inf_data['allele_number']
numerator = inf_data['numerator']
covar_int = inf_data['covar_int']
traj      = inf_data['traj']
mutants   = inf_data['mutant_sites']
locs      = inf_data['locations']
dates     = inf_data['times']
idx       = list(locs).index('europe-united kingdom-england_wales_scotland-2020-01-01-2021-01-01.n---2020-2-24-2021-1-1')
traj_uk   = traj[idx]
muts_uk   = mutants[idx]
time      = dates[idx]

q   = 5
N   = 10000
k   = 0.1
R   = 2
g   = 10
g  *= (N * k) / (1 + (k / R))

for i in range(len(covar_int)):
    covar_int[i, i] += g
    
start_t    = 188    # number of days after february 24th 2020 that the individuals infected with 20E-EU1 start migrating
end_t      = 288
T          = end_t - start_t + 1
uk_popsize = uk_popsize[uk_times.index(start_t):uk_times.index(end_t)+1]
traj_uk    = traj_uk[list(time).index(start_t):list(time).index(end_t)+1]

ref_seq, ref_tag  = dp.get_MSA(REF_TAG +'.fasta')
ref_seq       = list(ref_seq[0])
allele_number = pd.unique([int(i[:-2]) for i in alleles])
mut_number    = pd.unique([int(i[:-2]) for i in muts_uk])
ref_poly      = np.array(ref_seq)[allele_number]

alleles_sorted = np.argsort(allele_number)
positions      = np.searchsorted(allele_number[alleles_sorted], mut_number)

counts     = [0, 4, 8, 10, 20, 50, 100, 500, 1000]
s_tot      = []
for i in range(len(counts)):
    count = counts[i]
    inflow_seqs   = [[inflow_seq] for i in range(T)]
    inflow_counts = [[count] for i in range(T)]
    pop_in        = [np.sum(i) for i in inflow_counts]
    inflow_term   = dp.allele_counter_in(inflow_seqs, inflow_counts, traj_uk, pop_in, uk_popsize, k, R, T, d=5)
    inflow_tot    = np.zeros(len(alleles))
    for j in range(len(mut_number)):
        inflow_tot[positions[j] * 5 : (positions[j] + 1) * 5] += inflow_term[j * 5 : (j + 1) * 5]
    A = covar_int
    b = numerator - inflow_tot
    
    s = linalg.solve(A, b, assume_a='sym')

    # normalize reference nucleotide to zero
    L = int(len(s) / 5)
    selection  = np.reshape(s, (L, 5))

    s_new = []
    for i in range(L):
        idx = NUC.index(ref_poly[i])
        temp_s    = selection[i]
        temp_s    = temp_s - temp_s[idx]
        s_new.append(temp_s)
    selection = s_new
    selection = np.array(selection).flatten()
    s_tot.append(selection)
    np.savez_compressed(os.path.join(INF_DIR, f'infer-{DATA_DATE}-uk-inflow{count}.npz'), 
                        selection=selection, allele_number=allele_number)

SyntaxError: invalid syntax (<ipython-input-2-5241535f716a>, line 9)

In [314]:
EU1 = [['NSP16-199-2-C', 'NSP1-60-2-C', 'NSP3-1189-2-T', 'M-93-2-G', 'N-220-1-T', 'ORF10-30-0-T', 'S-222-1-T']]
np.save(os.path.join(DATA_DIR, '20E-EU1.npy'), EU1)

In [311]:
if not os.path.exists(os.path.join(INF_DIR, 'inflow-uk')):
    os.mkdir(os.path.join(INF_DIR, 'inflow-uk'))
for count in counts:
    shutil.copy(os.path.join(INF_DIR, f'infer-{DATA_DATE}-uk-inflow{count}.npz'), 
                os.path.join(INF_DIR, 'inflow-uk', f'infer-{DATA_DATE}-uk-inflow{count}.npz'))
    

In [333]:
if not os.path.exists(os.path.join(INF_DIR, 'inflow-uk')):
    os.mkdir(os.path.join(INF_DIR, 'inflow-uk'))
inf_data  = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}--2021-05-01-uk-g-10' + '.npz'), allow_pickle=True)
alleles   = inf_data['allele_number']
counts     = [0, 4, 8, 10, 20, 50, 100, 500, 1000]
for count in counts:
    data = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-uk-inflow{count}.npz'), allow_pickle=True)
    np.savez_compressed(os.path.join(INF_DIR, 'inflow-uk', f'infer-{DATA_DATE}-uk-inflow{count}.npz'),
                        selection=data, allele_number=alleles)


In [60]:
sites    = [dp.get_label(i[:-2]) + '-' + i[-1] for i in alleles]
var_idxs = np.array([sites.index(site) for site in link_sites])
s_var    = np.zeros(len(s_tot))
for i in range(len(var_idxs)):
    for j in range(len(s_tot)):
        s_var[j] += s_tot[j][var_idxs[i]]
slope  = (s_var[4] - s_var[0]) / (counts[4] - counts[0])
x_zero = - s_var[0] / slope
print('number of migrating individuals per day in order for 20E-EU1 variant to be zero', x_zero)

[ 0.15612642  0.15442725  0.15272809  0.1518785   0.14763058  0.13488683
  0.11364723 -0.05626954 -0.26866549]
number of migrating individuals per day in order for 20E-EU1 variant to be zero 367.53623582591405


### Inferring the effect of B.1.1.7 on selection coefficients

In [4]:
alpha = ['NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--',
         'NSP6-108-1--', 'NSP6-108-2--', 'S-501-0-T', 'NSP12-412-2-T', 'NSP2-36-2-T', 'NSP3-183-1-T', 'NSP3-890-1-A', 
         'NSP3-1089-2-T', 'NSP3-1412-1-C', 'NSP12-613-2-T', 'NSP12-912-2-C', 'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 
         'S-69-2--', 'S-70-0--', 'S-143-2--', 'S-144-0--', 'S-144-1--', 'S-570-1-A', 'S-681-1-A', 'S-716-1-T', 'S-982-0-G', 
         'S-1118-0-C', 'ORF8-27-0-T', 'ORF8-52-1-T', 'ORF8-73-1-G', 'N-3-0-C', 'N-3-1-T', 'N-3-2-A', 'N-235-1-T',
         'N-203-1-A', 'N-203-2-A', 'N-204-0-C']
np.save(os.path.join(SARS_DIR, 'b117-muts.npy'), alpha)

cutoff     = 0

count_data = np.load(os.path.join(DATA_DIR, 'nucleotide-counts.npz'), allow_pickle=True)
muts       = count_data['allele_number']
counts     = count_data['counts']

# Same as above but for a single file
syn_data  = np.load(os.path.join(DATA_DIR, 'synonymous-prot-test.npz'), allow_pickle=True)
data      = np.load(os.path.join(INF_DIR, f'inf-{DATA_DATE}', f'infer-{DATA_DATE}-g-40.npz'), allow_pickle=True)

NUC = ['-', 'A', 'C', 'G', 'T']
q   = 5
N   = 10000
k   = 0.1
R   = 2
g   = 40
alleles   = data['allele_number']
numerator = data['numerator']
covar_int = data['covar_int']
syn_muts  = syn_data['nuc_index']
types     = syn_data['types']
types     = types[np.isin(syn_muts, alleles)]
g1 = np.array(g, dtype=float) * (N * k) / (1 + (k / R))
for i in range(len(covar_int)):
    covar_int[i, i] += g1
    
mask   = np.isin(muts, alleles)
muts   = muts[mask]
counts = counts[mask]
    
# Mask out sites that have mutations in alpha
alpha_short = np.array([i[:-2] for i in alpha])
muts_short  = np.array([dp.get_label2(i)[:-2] for i in alleles])
mask        = np.isin(muts_short, alpha_short, invert=True)
alleles     = alleles[mask]
numerator   = numerator[mask]
covar_int   = covar_int[mask][:, mask]
types       = types[mask]
counts      = counts[mask]

# infer selection coefficients
s      = linalg.solve(covar_int, numerator, assume_a='sym')
s_ind  = numerator / np.diag(covar_int)
errors = 1 / np.sqrt(np.absolute(np.diag(covar_int)))

print('selection coefficients found')

# normalize reference nucleotide to zero
L = int(len(s) / 5)
selection  = np.reshape(s, (L, q))
selection_nocovar = np.reshape(s_ind, (L, q))
ref_seq, ref_tag  = dp.get_MSA(REF_TAG +'.fasta')
ref_seq  = list(ref_seq[0])
allele_number = np.unique([int(i[:-2]) for i in alleles])
ref_poly = np.array(ref_seq)[allele_number]

s_new = []
s_SL  = []
for i in range(L):
    idx = NUC.index(ref_poly[i])
    temp_s    = selection[i]
    temp_s    = temp_s - temp_s[idx]
    temp_s_SL = selection_nocovar[i]
    temp_s_SL = temp_s_SL - temp_s_SL[idx]
    s_new.append(temp_s)
    s_SL.append(temp_s_SL)
selection         = s_new
selection_nocovar = s_SL

selection         = np.array(selection).flatten()
selection_nocovar = np.array(selection_nocovar).flatten()
error_bars        = errors

# Eliminate reference nucleotides and mutations that aren't observed

mask      = np.nonzero(np.logical_and(selection!=0, counts>cutoff))[0]
alleles   = alleles[mask]
selection = selection[mask]
s_ind     = selection_nocovar[mask]
errors    = error_bars[mask]

In [6]:
traj_data = np.load(os.path.join(DATA_DIR, 'trajectories-1pct.npz'), allow_pickle=True)
traj      = traj_data['traj']
mutants   = traj_data['mutant_sites']
muts_new  = []
for i in range(len(mutants)):
    #idxs_keep  = np.array([j for j in range(len(mutants[i])) if mutants[i][j] in alleles])
    idxs_keep  = np.isin(mutants[i], alleles)
    muts_new.append(np.array(mutants[i])[idxs_keep])
    traj[i]    = np.array([traj[i][:, j] for j in range(len(traj[i][0])) if j in idxs_keep])
    traj[i]    = np.swapaxes(traj[i], 0, 1)
mutants = muts_new

np.savez_compressed(os.path.join(INF_DIR, f'infer-{DATA_DATE}-g-40-observed-mask-alpha.npz'),
                    selection=selection, selection_independent=s_ind, error_bars=errors,
                    allele_number=alleles, traj=traj, mutant_sites=mutants)
    
dp.website_file2(os.path.join(INF_DIR, f'infer-{DATA_DATE}-g-40-observed-mask-alpha.npz'),
                 os.path.join(DATA_DIR, 'synonymous-prot-test.npz'),
                 os.path.join(DATA_DIR, 'linked-sites.npy'), 
                 os.path.join(DATA_DIR, f'selection-g-40-observed-mask-alpha.csv'))

NameError: name 'reg' is not defined

In [20]:
unmasked    = np.load(os.path.join(INF_DIR, f'inf-{DATA_DATE}', f'infer-{DATA_DATE}-g-40.npz'), allow_pickle=True)
masked      = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-g-{g}-observed-mask-alpha.npz'), allow_pickle=True)
s_unmask    = unmasked['selection']
s_mask      = masked['selection']
muts_mask   = [dp.get_label2(i) for i in masked['allele_number']]
muts_unmask = [dp.get_label2(i) for i in unmasked['allele_number']]
link_sites  = ['ORF10-30-0-T', 'N-220-1-T', 'M-93-2-G', 'S-222-1-T', 'NSP16-199-2-C', 'NSP3-1189-2-T', 'NSP1-60-2-C']

sel_mask    = np.sum(s_mask[np.isin(muts_mask, link_sites)])
sel_unmask  = np.sum(s_unmask[np.isin(muts_unmask, link_sites)])
print('selection coefficient for 20E-EU1 when alpha is kept in the data:                               ', sel_unmask)
print('selection coefficient for 20E-EU1 when alpha mutations are reverted to the reference nucleotide:', sel_mask)

selection coefficient for 20E-EU1 when alpha is kept in the data:                                0.10225873676301382
selection coefficient for 20E-EU1 when alpha mutations are reverted to the reference nucleotide: -0.01673100542328889


# Calculating Statistics

In [127]:
# Finding the correlation between frequency changes for B.1.1.7 and 20E-EU1
traj_file = os.path.join(DATA_DIR, 'linked-trajectories-1pct.csv')

print('correlation of frequency changes:', dp.freq_change_correlation(traj_file, 'alpha', '20E_EU1'))

print('correlation of frequency changes in UK:', dp.freq_change_correlation(traj_file, 'alpha', '20E_EU1', 
                                                                            region='europe-united kingdom-england_wales_scotland-2020-01-01-2021-01-01.n---2020-2-24-2021-1-1'))

print('correlation of frequency changes in UK:', dp.freq_change_correlation(traj_file, 'alpha', '20E_EU1', 
                                                                            region=['europe-united kingdom-england_wales_scotland-2020-01-01-2021-01-01.n---2020-2-24-2021-1-1',
                                                                                    'europe-united kingdom-england_wales_scotland-2021-01-01-2021-05-01.n---2020-12-29-2021-5-1']))


74
74
correlation of frequency changes: -0.2639345794687149
1
1
correlation of frequency changes in UK: -0.7460799758819553
2
2
correlation of frequency changes in UK: -0.7332002054076147


# Finding Null Distribution

In [ ]:
reload(epi_figs)
null = epi_figs.find_null_distribution_ind(os.path.join(INF_DIR, 'infer-2021-01-31-tv.npz'), 
                                           os.path.join(DATA_DIR, 'linked-sites.npy'),
                                           os.path.join(SARS_DIR, 'individual-regions', 'inference')
fig, axes = plt.subplots(1,1, figsize=[20,10])
axes.hist(null, log=True, bins=50)

f = open(os.path.join(SARS_DIR, 'null-distribution-new.csv'), mode='w')
for i in range(len(null)):
    f.write('%.8f\n' % null[i])
f.close()


# Preliminary Estimate of Omicron Selection

In [ ]:
muts    = ['NC-240-T', 'NSP3-38-1-G', 'NSP3-106-2-T', 'NSP3-889-2-G', 'NSP3-1265-1--', 'NSP3-1265-2--', 
           'NSP3-1266-0--', 'NSP3-1892-0-A', 'NSP4-492-1-T', 'NSP5-132-1-A', 'NSP6-104-1--', 'NSP6-104-2--', 
           'NSP6-105-0--', 'NSP6-105-1--', 'NSP6-105-2--', 'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 
           'NSP6-107-0--', 'NSP6-189-0-G', 'NSP10-57-2-C', 'NSP12-323-1-T', 'NSP12-600-2-T', 'NSP14-42-0-G', 
           'S-67-1-T', 'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 'S-69-2--', 'S-70-0--', 'S-95-1-T', 
           'S-142-1--', 'S-142-2--', 'S-143-0--', 'S-143-1--', 'S-143-2--', 'S-144-0--', 'S-144-1--', 
           'S-144-2--', 'S-145-0--', 'S-211-1--', 'S-211-2--', 'S-212-0--', 'S-339-1-A', 'S-371-0-C', 
           'S-371-1-T', 'S-373-0-C', 'S-417-2-T', 'S-440-2-G', 'S-446-0-A', 'S-547-1-A', 'S-614-1-G', 
           'S-655-0-T', 'S-679-2-G', 'S-681-1-A', 'S-764-2-A', 'S-796-0-T', 'S-856-2-A', 'S-954-2-T', 
           'S-969-2-A', 'S-981-0-T', 'S-1146-2-T', 'ORF3a-64-2-T', 'E-9-1-T', 'M-3-1-G', 'M-19-0-G', 
           'M-63-0-A', 'ORF6-20-0-C', 'ORF7b-18-2-T', 'NC-28270-T', 'N-13-1-T', 'N-30-1--', 'N-30-2--', 
           'N-31-0--', 'N-31-1--', 'N-31-2--', 'N-32-0--', 'N-32-1--', 'N-32-2--', 'N-33-0--', 'N-203-1-A', 
           'N-203-2-A', 'N-204-0-C']    # Omicron mutations (excluding the insertion, which doesn't appear in our data)
data    = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-g-40-1pct-observed.npz'), allow_pickle=True)
alleles = [dp.get_label2(i) for i in data['allele_number']]
s       = data['selection']
s_tot   = np.sum(s[np.isin(alleles, muts)])
print(f'selection for omicron variant is {s_tot}')
print(f'number of mutations showing up in the old data is {len(s[np.isin(alleles, muts)])} out of {len(muts) + 9} total mutations')

# Early Detection

__Early detection in the UK__

In [4]:
# infering the selection coefficients using a cutoff frequency of 5% in the UK
input_dir     = os.path.join(SSH_DATA, 'data', 'uk')
freq_script   = 'epi-covar-parallel.py'
archive_loc   = os.path.join(SCRIPT_DIR, 'Archive-tv-int')
temp_dir      = os.path.join(SSH_DATA, 'data', 'freqs-tv')
uk_dir        = os.path.join(SSH_DATA, 'data', 'uk')
freqs_dir     = os.path.join(SSH_DATA, 'data', 'freq_0.05')

# run on cluster
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
print(f'mkdir {uk_dir}')
print(f'cp \"{freqs_dir}/europe-united kingdom-england_wales_scotland*\" \"{uk_dir}\"')
print('')

num_files     = 3    # the number of data files that must be read from the directory

job_file1     = 'job-find-freqs-tv.sh'
job_str1      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=250G
#SBATCH --time=7-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq-tv-error-%a
#SBATCH -o ./MPL/out/freq-tv-out-%a
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)
module purge
module load anaconda3

cd ./Archive-tv-int
g++ src/main-test.cpp src/inf-test.cpp src/io-test.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11
pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python ../{freq_script} --data \"$file\" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {SCRATCH} --timed 1 --tv_inference --delta_t 10
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, freq_script, SSH_HOME))
print('scp -r %s %s. &&'   % (archive_loc, SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file1, SSH_HOME))
print('')

# run on cluster
print('sbatch %s &&' % job_file1)
print('sbatch %s' % job_file2)
print(f'mkdir {SSH_DATA}/inf-{DATA_DATE}-uk-tv')
print(f'mv {out_file}---* {SSH_DATA}/inf-{DATA_DATE}-uk-tv')
print('')

# transfer files from cluster
print(f'scp -r {SSH_HOME}{temp_dir} {DATA_DIR}')

ls /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/uk | wc -l
mkdir /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/uk
cp "/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/freq_0.05/europe-united kingdom-england_wales_scotland*" "/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14/uk"

scp /Users/brianlee/Python/MPL/epi-covar-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/Python/MPL/epi-inf-parallel-tv.py blee098@cluster.hpcc.ucr.edu:. &&
scp -r /Users/brianlee/Python/MPL/6-29-20-epidemiological/Archive-tv-int blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-find-freqs-tv.sh blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-infer-tv.sh blee098@cluster.hpcc.ucr.edu:.

sbatch job-find-freqs-tv.sh &&
sbatch job-infer-tv.sh
mkdir /rhome/blee098/bigdata/SARS-CoV-2-Data/inf-2021-08-14

In [ ]:
local_freqs = os.path.join(DATA_DIR, 'freqs-tv')
t_start = []
t_end   = []
for file in os.listdir(local_freqs):
    if os.path.isfile(os.path.join(local_freqs, file)):
        times = np.load(os.path.join(local_freqs, file), allow_pickle=True)['times_full']
        t_start.append(times[0])
        t_end.append(times[-1])
t_start = np.amin(t_start)
t_end   = np.amax(t_end)

In [22]:
inf_script    = 'epi-inf-parallel-tv.py'
temp_dir      = os.path.join(SSH_DATA, 'data', 'freqs-tv')
out_file      = os.path.join(SSH_DATA, f'infer-tv-uk-{DATA_DATE}')

# Break into time intervals so inference on different dates can be run in parallel
start_times   = np.arange(t_start,     t_end - 10, 10)   
end_times     = np.arange(t_start + 9, t_end - 1,  10)

s_times       = ' '.join([str(i) for i in start_times])
e_times       = ' '.join([str(i) for i in end_times])
print(s_times)    # copy paste this under s_times in the job_string
print(e_times)    # copy paste this under e_times in the job_string

job_file2 = 'job-infer-tv.sh'
job_str2  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=50G
#SBATCH --time=2-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/inf-tv2-error-%a
#SBATCH -o ./MPL/out/inf-tv2-out-%a
#SBATCH --array=0-{len(start_times) - 1}
s_times=(60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560)
e_times=(69 79 89 99 109 119 129 139 149 159 169 179 189 199 209 219 229 239 249 259 269 279 289 299 309 319 329 339 349 359 369 379 389 399 409 419 429 439 449 459 469 479 489 499 509 519 529 539 549 559 569)

module purge
module load anaconda3

init_time=${{s_times[$SLURM_ARRAY_TASK_ID]}}
final_time=${{e_times[$SLURM_ARRAY_TASK_ID]}}
python {inf_script} --data {temp_dir} --timed 1 -o {out_file} -q 5 --g1 10 --delta_t 10 --start_time $init_time --end_time $final_time --no_traj
"""

f = open(os.path.join(SCRIPT_DIR, job_file2), 'w')
f.write('%s\n' % job_str2)
f.close()

# run on cluster
print(f'mkdir {SSH_DATA}/inf-{DATA_DATE}-uk-tv')
print(f'mv {out_file}---* {SSH_DATA}/inf-{DATA_DATE}-uk-tv')
print('')

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, inf_script, SSH_HOME))
print('scp -r %s %s. &&'   % (archive_loc, SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file2, SSH_HOME))
print('')

# transfer inference files to local directory
print(f'scp -r {SSH_HOME}{SSH_DATA}/inf-{DATA_DATE}-uk-tv {INF_DIR}')

60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560
69 79 89 99 109 119 129 139 149 159 169 179 189 199 209 219 229 239 249 259 269 279 289 299 309 319 329 339 349 359 369 379 389 399 409 419 429 439 449 459 469 479 489 499 509 519 529 539 549 559 569
mkdir /rhome/blee098/bigdata/SARS-CoV-2-Data/inf-2021-08-14-uk-tv2
mv /rhome/blee098/bigdata/SARS-CoV-2-Data/infer-tv-uk-2021-08-14-new---* /rhome/blee098/bigdata/SARS-CoV-2-Data/inf-2021-08-14-uk-tv2

scp /Users/brianlee/Python/MPL/epi-inf-parallel-tv.py blee098@cluster.hpcc.ucr.edu:. &&
scp -r /Users/brianlee/Python/MPL/6-29-20-epidemiological/Archive-tv-int blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-infer-tv4.sh blee098@cluster.hpcc.ucr.ed

In [145]:
alleles    = []
selection  = []
times_temp = []
for file in os.listdir(os.path.join(INF_DIR, f'inf-{DATA_DATE}-uk-tv')):
    times_temp.append(int(file.split('---')[-1][:-4]))
times = np.arange(np.amin(times_temp), np.amax(times_temp) + 1)

for i in times:
    data_temp = np.load(os.path.join(INF_DIR, f'inf-{DATA_DATE}-uk-tv', f'infer-tv-uk-{DATA_DATE}---{i}.npz'))
    if 'allele_number' in data_temp:
        alleles = data_temp['allele_number']
    selection.append(data_temp['selection'].flatten())
np.savez_compressed(os.path.join(INF_DIR, f'infer-{DATA_DATE}-uk-tv.npz'), selection=selection, allele_number=alleles, times=times)
inf_times = times

(506, 36610)


In [ ]:
# find uk frequency trajectories
input_dir     = os.path.join(SSH_DATA, 'data', 'uk')
freq_script   = 'epi-traj-parallel.py'
temp_dir      = os.path.join(SSH_DATA, 'data', 'freqs-uk')

# run on cluster
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
print('')

num_files     = 3    # the number of data files that must be read from the directory

job_file1     = 'job-find-freqs-uk.sh'
job_str1      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=150G
#SBATCH --time=5-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq-uk-error-%a
#SBATCH -o ./MPL/out/freq-uk-out-%a
#SBATCH --array=0-{num_files - 1}

files=({input_dir}/*)
module purge
module load anaconda3

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python {freq_script} --data \"$file\" -o $tempout -q 5 --timed 1 --window 10
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()


# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, freq_script, SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file1, SSH_HOME))
print('')

# run on cluster
print(f'mkdir {temp_dir}')
print('sbatch %s &&' % job_file1)
print('')

# transfer files from cluster
print(f'scp -r {SSH_HOME}{temp_dir} {DATA_DIR}')

In [146]:
mutant_sites = []
traj         = []
traj_full    = []
locs         = []
times_full   = []
times        = []
locations    = []
for file in os.listdir(os.path.join(DATA_DIR, 'freqs-uk')):
    loc  = file[:-4]
    data = np.load(os.path.join(DATA_DIR, 'freqs-uk', file), allow_pickle=True)
    traj.append(data['traj'])
    mutant_sites.append(data['allele_number'])
    traj_full.append(data['traj_nosmooth'])
    times_full.append(data['times_full'])
    times.append(data['times'])
    locations.append(loc)
    traj_times = data['times']
    if inf_times[-1] in traj_times:
        traj_times = traj_times[:list(traj_times).index(inf_times[-1])]

np.savez_compressed(os.path.join(INF_DIR, 'uk-trajectories.npz'), traj=traj, 
                    traj_nosmooth=traj_full, times=times, times_full=times_full, mutant_sites=mutant_sites,
                    locations=locations)

inf_data  = np.load(os.path.join(INF_DIR, f'infer-{DATA_DATE}-uk-tv.npz'))
selection = inf_data['selection']
alleles   = inf_data['allele_number']
times_inf = inf_data['times']

np.savez_compressed(os.path.join(INF_DIR, f'infer-{DATA_DATE}-uk-tv-traj.npz'),
                    selection=selection, allele_number=alleles, traj=traj, mutant_sites=mutant_sites, traj_nosmooth=traj_full,
                    times=times, times_full=times_full, times_inf=times_inf, locations=locations)
times = times_inf

(26715,)
(309, 26715)
(27630,)
(81, 27630)
(33175,)
(120, 33175)
['europe-united kingdom-england_wales_scotland-2020-01-01-2021-01-01.n---2020-2-24-2021-1-1', 'europe-united kingdom-england_wales_scotland-2021-05-01-2021-09-01.n---2021-5-1-2021-7-24', 'europe-united kingdom-england_wales_scotland-2021-01-01-2021-05-01.n---2020-12-29-2021-5-1']


In [2]:
twentyE_EU1 = ['NSP16-199-2-C', 'NSP1-60-2-C', 'NSP3-1189-2-T', 'M-93-2-G', 'N-220-1-T', 'ORF10-30-0-T', 'S-222-1-T']    # Done

delta       = ['NSP12-671-0-A', 'NC-209-T', 'NSP13-77-1-T', 'S-19-1-G', 'S-156-1--', 'S-156-2--', 'S-157-0--', 'S-157-1--', 'S-157-2--', 
               'S-158-0--', 'S-478-1-A', 'S-950-0-A', 'ORF3a-26-1-T', 'M-82-1-C', 'ORF7a-82-1-C', 'ORF7a-120-1-T', 
               'ORF8-119-0--', 'ORF8-119-1--', 'ORF8-119-2--', 'ORF8-120-0--', 'ORF8-120-1--', 'ORF8-120-2--', 'N-63-1-G', 
               'N-377-0-T', 'S-452-1-G', 'S-681-1-G']  
# Eliminated 'N-203-1-T' and 'NC-28270--' 

tv_inf_file = os.path.join(INF_DIR, f'infer-{DATA_DATE}-uk-tv-traj.npz')
out_file    = os.path.join(DATA_DIR, 'delta-tv-s')
dp.save_traj_selection(tv_inf_file, group=delta, traj_site='S-19-1-G', out_file=out_file)

out_file    = os.path.join(DATA_DIR, '20E-EU1-tv-s')
dp.save_traj_selection(tv_inf_file, group=twentyE_EU1, traj_site='S-222-1-T', out_file=out_file)

out_file    = os.path.join(DATA_DIR, 'S-A222V-tv-s')
dp.save_traj_selection(tv_inf_file, group=['S-222-1-T'], traj_site='S-222-1-T', out_file=out_file)

__Early Detection in London__

In [ ]:
london_dir   = os.path.join(SSH_DATA, 'data' + '-lond')
london_local = DATA_DIR + '-lond'

In [ ]:
# extracting only the data in the United Kingdom in London
identifier_uk = f'regions-{DATA_DATE}-lond'

selected_uk   =  [['europe', 'united kingdom', 'england', 'lond', None, None]]

f = open(os.path.join(SARS_DIR, identifier_uk + '.npy'), mode='w+b')
np.save(f, selected_uk)
f.close()


# Transfer scripts, run processing, and extract files
# Processing clips the time series in different regions according to set rules regarding the distribution of the number of sampled genomes over time
data_module   = 'data_processing.py'
data_mod_path = os.path.join(SCRIPT_DIR, data_module)
out_folder    = os.path.join(SSH_DATA, 'data' + '-lond')
input_file    = 'merged-final.csv'
script_file   = 'processing-multiallele.py'

job_file      = 'job-processing-uk.sh'
job_str       = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=200G
#SBATCH --time=10:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/genome-uk-error
#SBATCH -o ./MPL/out/genome-uk-out
#SBATCH -p highmem

module purge
module load anaconda3

python {script_file} --input_file {os.path.join(SSH_DATA, input_file)} -o {out_folder} --regions {identifier_uk + '.npy'} --find_syn_off --no_trim
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

print('scp %s/%s %s. &&' % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s. &&' % (SCRIPT_DIR, data_module, SSH_HOME))
print('scp %s/%s %s. &&' % (SARS_DIR, identifier_uk + '.npy', SSH_HOME))
print('scp %s/%s %s.'    % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

print('sbatch %s' % job_file)
print('')

print('scp -r %s%s %s' % (SSH_HOME, out_folder, SARS_DIR))

In [ ]:
# Finding time series with good sampling in London
input_dir     = os.path.join(SSH_DATA, 'data' + '-lond')
output_dir    = input_dir
script_file   = 'trim-sampling-intervals.py'
trim_dir      = 'genome-trimmed'

job_file      = 'job-trim-intervals-uk.sh'
job_str       = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=200G
#SBATCH --time=20:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/sampling-uk-error
#SBATCH -o ./MPL/out/sampling-uk-out
#SBATCH -p batch

module purge
module load anaconda3

python {script_file} --input_dir {input_dir} -o {output_dir} --freq_cutoff --trim_dir {trim_dir} --min_seqs 20 --window 7
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()


print('scp %s/%s %s. &&'   % (SCRIPT_DIR, script_file, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file, SSH_HOME))
print('')

print('sbatch %s' % job_file)

In [ ]:
# Trim the data file in London
# This cuts the data file to dates that have good sampling. The end point is arbitrary, but eliminating data before 2020-10-23 is important.
# The "days" parameter in the bash script determines the number of days eliminated at the beginning of the time series.
input_dir     = os.path.join(SSH_DATA, 'data' + '-lond', 'freq_0.05', 'europe-united kingdom-england-lond---2020-9-23-2021-5-27.npz')
freq_script   = 'cut-file.py'
out_file      = os.path.join(SSH_DATA, 'data' + '-lond', 'freq_0.05', 'europe-united kingdom-england-lond---2020-10-23-2021-5-27.npz')

job_file1     = 'job-cut-series.sh'
job_str1      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --time=4-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/cut-lond-error
#SBATCH -o ./MPL/out/cut-lond-out
#SBATCH -p batch

module purge
module load anaconda3

python {freq_script} --input_dir \"{input_dir}\" -o \"{out_file}\" --days 30
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()

print(f'scp {SCRIPT_DIR}/{freq_script} {SSH_HOME} &&')
print(f'scp {SCRIPT_DIR}/{job_file1} {SSH_HOME}')

print('')
print(f'sbatch {job_file1}')
print(f'rm "{input_dir}"')

print('')
print('scp -r %s%s %s' % (SSH_HOME, output_dir, SARS_DIR))

In [1]:
# infering the selection coefficients using a cutoff frequency of 5% in London
input_dir     = os.path.join(SSH_DATA, 'data' + '-lond', 'freq_0.05')
freq_script   = 'epi-covar-parallel.py'
archive_loc   = os.path.join(SCRIPT_DIR, 'Archive-tv-int')
temp_dir      = os.path.join(SSH_DATA, 'data' + '-lond', 'freqs-tv')

# run on cluster
print(f'mkdir {temp_dir}')
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
print('')

num_files     = 3    # the number of data files that must be read from the directory

job_file1     = 'job-find-freqs-tv-lond.sh'
job_str1      = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --time=4-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/freq-tv-lond-error-%a
#SBATCH -o ./MPL/out/freq-tv-lond-out-%a
#SBATCH --array=0-{num_files - 1}
#SBATCH -p batch

files=({input_dir}/*)
shein
module purge
module load anaconda3


cd ./Archive-tv-int
g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11
pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python ../{freq_script} --data \"$file\" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {SCRATCH} --timed 1 --tv_inference --window 15 --delta_t 15
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, freq_script, SSH_HOME))
print('scp -r %s %s. &&'   % (archive_loc, SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file1, SSH_HOME))
print('')

# run on cluster 
print('sbatch %s' % job_file1)
print('')

# transfer files from cluster
print(f'scp -r {SSH_HOME}{temp_dir} {london_dir}')

NameError: name 'os' is not defined

In [ ]:
local_freqs = os.path.join(london_local, 'freqs-tv')
t_start = []
t_end   = []
for file in os.listdir(local_freqs):
    if os.path.isfile(os.path.join(local_freqs, file)):
        times = np.load(os.path.join(local_freqs, file), allow_pickle=True)['times_full']
        t_start.append(times[0])
        t_end.append(times[-1])
t_start = np.amin(t_start)
t_end   = np.amax(t_end)

In [ ]:
inf_script    = 'epi-inf-parallel-tv.py'
out_file      = os.path.join(SSH_DATA, f'infer-lond-tv-{DATA_DATE}')
start_times   = np.arange(t_start,     t_end - 10, 10)   
end_times     = np.arange(t_start + 9, t_end - 1,  10)

s_times       = ' '.join([str(i) for i in start_times])
e_times       = ' '.join([str(i) for i in end_times])
print(s_times)    # copy paste this under s_times in the job_string
print(e_times)    # copy paste this under e_times in the job_string


job_file2 = 'job-infer-tv-lond.sh'
job_str2  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --time=2-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/inf-tv-lond-error-%a
#SBATCH -o ./MPL/out/inf-tv-lond-out-%a
#SBATCH --array=0-17
#SBATCH -p batch
s_times=(300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450)
e_times=(309 319 329 339 349 359 369 379 389 399 409 419 429 439 449 459)

module purge
module load anaconda3

init_time=${{s_times[$SLURM_ARRAY_TASK_ID]}}
final_time=${{e_times[$SLURM_ARRAY_TASK_ID]}}
python {inf_script} --data {temp_dir} --timed 2 -o {out_file} -q 5 --g1 10 --start_time $init_time --end_time $final_time --delta_t 15
"""

f = open(os.path.join(SCRIPT_DIR, job_file2), 'w')
f.write('%s\n' % job_str2)
f.close()

# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, inf_script, SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s.'      % (SCRIPT_DIR, job_file2, SSH_HOME))
print('')

# run on cluster
print(f'mkdir {out_file}')
print('sbatch %s' % job_file2)
print(f'mv {out_file}---* {SSH_DATA}/infer-lond-tv-{DATA_DATE}')
print('')

# transfer files from cluster
print('scp -r %s%s %s'      % (SSH_HOME, out_file, INF_DIR + '-lond'))

In [166]:
### TRAJECTORIES (unnormalized) ###

input_dir     = os.path.join(SSH_DATA, 'data' + '-lond', 'freq_0.05')
count_script  = 'epi-traj-parallel.py'
temp_dir      = os.path.join(SSH_DATA, 'data' + '-lond', 'traj')

# run on cluster to find number of files
print(f'ls {input_dir} | wc -l')    # in order to find the number of files in the input directory
num_files = 2    # the number of data files 

job_file1 = 'job-traj-lond.sh'
job_str1  = f"""#!/bin/bash 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --time=2-00:00:00
#SBATCH --mail-user={USER_NAME}{EMAIL_EXT}
#SBATCH --mail-type=ALL
#SBATCH -e ./MPL/out/traj-lond-error-%a
#SBATCH -o ./MPL/out/traj-lond-out-%a
#SBATCH --array=0-{num_files - 1}
#SBATCH -p batch

files=({input_dir}/*)

module purge
module load anaconda3

pwd

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python {count_script} --data \"$file\" -o $tempout --timed 1 --window 15
""" 

f = open(os.path.join(SCRIPT_DIR, job_file1), 'w')
f.write('%s\n' % job_str1)
f.close()


# transfer files to cluster
print('scp %s/%s %s. &&'   % (INF_SCRIPTS, 'epi-traj-parallel.py', SSH_HOME))
print('scp %s %s. &&'      % (os.path.join(SCRIPT_DIR, 'data_processing.py'), SSH_HOME))
print('scp %s/%s %s. &&'   % (SCRIPT_DIR, job_file1, SSH_HOME))
print('')

# run on cluster
print('sbatch %s' % job_file1)
print('')

print(f'scp -r {SSH_HOME}{temp_dir} {london_local}')

ls /rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14-lond/freq_0.05_new | wc -l
scp /Users/brianlee/Python/MPL/epi-traj-parallel.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/data_processing.py blee098@cluster.hpcc.ucr.edu:. &&
scp /Users/brianlee/SARS-CoV-2-Data/Processing-files/job-traj-lond.sh blee098@cluster.hpcc.ucr.edu:. &&

sbatch job-traj-lond.sh

scp -r blee098@cluster.hpcc.ucr.edu:/rhome/blee098/bigdata/SARS-CoV-2-Data/2021-08-14-lond/traj /Users/brianlee/SARS-CoV-2-Data/2021-08-14-lond


In [167]:
# Combining inference files at different times into a single file
alleles    = []
selection  = []
times_temp = []
for file in os.listdir(os.path.join(INF_DIR + '-lond', f'infer-lond-tv-{DATA_DATE}')):
    times_temp.append(int(file.split('---')[-1][:-4]))
times = np.arange(np.amin(times_temp), np.amax(times_temp) + 1)

for i in times:
    data_temp = np.load(os.path.join(INF_DIR + '-lond', f'infer-lond-tv-{DATA_DATE}', f'infer-lond-tv-{DATA_DATE}---{i}.npz'))
    if 'allele_number' in data_temp:
        alleles = data_temp['allele_number']
    selection.append(data_temp['selection'].flatten())
np.savez_compressed(os.path.join(london_local, f'infer-lond-tv-{DATA_DATE}.npz'), selection=selection, allele_number=alleles, times=times)
inf_times = times

(151, 1345)


In [169]:
# Combine trajectory files
traj      = []
muts      = []
times     = []
locs      = []
t_full    = []
traj_full = []
traj_dir  = os.path.join(london_local, 'traj')
for file in os.listdir(traj_dir):
    data = np.load(os.path.join(traj_dir, file), allow_pickle=True)
    times.append(data['times'])
    muts.append(data['allele_number'])
    traj.append(data['traj'])
    locs.append(file[:-4])

np.savez_compressed(os.path.join(london_local, 'trajectories.npz'),
                    times=times, mutant_sites=muts, traj=traj, locations=locs, 
                    times_nosmooth=t_full, traj_nosmooth=traj_full)

inf_data  = np.load(os.path.join(INF_DIR + '-lond', f'infer-lond-tv-{DATA_DATE}.npz'), allow_pickle=True)
alleles   = inf_data['allele_number']
selection = inf_data['selection']
inf_times = inf_data['times']

np.savez_compressed(os.path.join(INF_DIR + '-lond', f'infer-lond-tv-{DATA_DATE}-traj.npz'), allele_number=alleles,
                    selection=selection, traj=traj, locations=locs, times=times, mutant_sites=muts,
                    times_inf=inf_times)

(153, 1345)


In [172]:
alpha          = ['NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--',
                  'NSP6-108-1--', 'NSP6-108-2--', 'S-501-0-T', 'NSP12-412-2-T', 'NSP2-36-2-T', 'NSP3-183-1-T', 'NSP3-890-1-A', 
                  'NSP3-1089-2-T', 'NSP3-1412-1-C', 'NSP12-613-2-T', 'NSP12-912-2-C', 'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 
                  'S-69-2--', 'S-70-0--', 'S-143-2--', 'S-144-0--', 'S-144-1--', 'S-570-1-A', 'S-681-1-A', 'S-716-1-T', 'S-982-0-G', 
                  'S-1118-0-C', 'ORF8-27-0-T', 'ORF8-52-1-T', 'ORF8-73-1-G', 'N-3-0-C', 'N-3-1-T', 'N-3-2-A', 'N-235-1-T']
tv_inf_file = os.path.join(INF_DIR, f'infer-lond-tv-{DATA_DATE}-traj.npz')
out_file    = os.path.join(DATA_DIR, 'alpha-tv-s')
dp.save_traj_selection(tv_inf_file, group=alpha, traj_site='S-570-1-A', out_file=out_file)